## AIHub Search Strange Source in Json

### Development Environment

In [ ]:
%pip install kss==3.7.3

KSS Argument Error: Restart Jupyter Kernel Runtime

In [ ]:
%pip install python-mecab-ko

KSS 3.7.3 matches python-mecab-ko

In [ ]:
%pip install pandas

In [1]:
import re
import os
import kss
import json
from mecab import MeCab
import pandas as pd
from glob import glob
from pprint import pprint

In [2]:
pwd

'd:\\AIHUB'

### Function

In [ ]:
def sorted_list(path_list):
    path_list = sorted(path_list, reverse=False)
    path_list = sorted(path_list, key=len)
    
    return path_list

In [3]:
def json_file_name_list(path_list):
    
    file_name  = [glob(i, recursive = True) for i in path_list][0]
    file_name = sorted_list(file_name)
    
    return file_name

In [4]:
def train_valid_json_file_name_list(path_list):

  train_file_name, valid_file_name = [glob(i, recursive = True) if 'rain' in i
                                      else glob(i, recursive = True)
                                      for i in path_list]

  train_file_name = sorted_list(train_file_name)
  valid_file_name = sorted_list(valid_file_name)
    
  return train_file_name, valid_file_name

In [ ]:
def divide_source_file_list(l, n): 
    
  for i in range(0, len(l), n): 
    yield l[i:i + n] 

In [6]:
def formal_preprocessing_text(source):
    preprocessing_sentence_list = []
    
    source = source.strip()
    # strip으로 앞뒤 공백 제거

    source = re.sub(r"\[.*?\]|\{.*?\}|\<.*?\>", "", source)
    # 기타 괄호 제거할 시 괄호 내부에 모든 텍스트 제거
    
    for sentence in kss.split_sentences(source, use_heuristic=False,
                                        num_workers=32):
    # KSS(Korean Sentence Segmentation)로 문장 분리 
    # Formal articles (wiki, news, essays): recommend to False

        if re.search("^[A-Za-z0-9ㄱ-ㅎ가-힣一-鿕㐀-䶵豈-龎①②③④⑤⑥⑦⑧⑨⑩⑪⑫⑬⑭⑮]", sentence[0]) is not None and \
            bool(re.match(r'[.]|[!]|[?]', sentence[-1])) == True and \
            len(sentence.split()) > 5:
            # 문장의 시작이 특수문자인 문장(영어 대소문자, 한글, 한자, 숫자, -, +, 
            # ①, ②, ③, ④, ⑤, ⑥, ⑦, ⑧, ⑨, ⑩, ⑪, ⑫, ⑬, ⑭, ⑮ 제외) 제외
            # 문장의 끝이 온점(.). 느낌표(!). 물음표(?)가 아닌 문장 제외
            # 다섯 어절 이하 문장 제외
            
                                    
            if ']' in sentence and '[' not in sentence:
                sentence  = re.sub(r".*?]", "", sentence)    
            # 중괄호 앞에 있는 '성명/직함]' 형태 제거
            
            sentence = re.sub(r"[^A-Za-z0-9ㄱ-ㅎ가-힣一-鿕㐀-䶵豈-龎()+-.,]", " ", sentence)
            # 특수문자 제거(영어 대소문자, 한글, 한자, 숫자, -, +, 소괄호 제외)

            sentence = sentence.strip()
            # strip으로 앞뒤 공백 제거
            
            total_length = len(sentence.replace(" " , ""))
            hangeul_length = len(re.sub(r"[^ㄱ-ㅣ가-힣\s]", "", sentence.replace(" " , "")))
            hangeul_ratio = hangeul_length / total_length
            if hangeul_ratio >= 0.5:
            # 한글이 아닌 문자열이 50% 이상이 넘은 문장 제외
                    
                for sentence2 in kss.split_sentences(sentence, use_heuristic=False,
                                        num_workers=32):
                    for sentence3 in kss.split_sentences(sentence2, use_heuristic=False,
                                                         num_workers=32):
                        preprocessing_sentence_list.append(sentence3)
                        
            # 마지막에 KSS(Korean Sentence Segmentation)로 문장 분리 2번 실행

  
    return preprocessing_sentence_list

### AIHUB 전문분야 말뭉치

In [9]:
path_list = ['AIHUB_전문분야 말뭉치/Training/'+ '/**/*.json', 
             'AIHUB_전문분야 말뭉치/Validation/'+ '/**/*.json']
train_file_name, valid_file_name = train_valid_json_file_name_list(path_list)

In [501]:
train_file_name[1]

'AIHUB_전문분야 말뭉치/Training\\[원천]전문분야_train\\논문_1_1.json'

In [502]:
train_file_name[10]

'AIHUB_전문분야 말뭉치/Training\\[원천]전문분야_train\\특허_03.json'

In [526]:
train_file_name[24]

'AIHUB_전문분야 말뭉치/Training\\[원천]전문분야_train\\특허_z_1.json'

In [519]:
train_file_name[2]

'AIHUB_전문분야 말뭉치/Training\\[원천]전문분야_train\\법령_l_1000_1.json'

In [10]:
train_file_name[30]

'AIHUB_전문분야 말뭉치/Training\\[원천]전문분야_train\\판례_case03.json'

In [527]:
source_file = train_file_name[1]

with open(source_file, 'r', encoding='utf-8') as one_json_file:
    one_json_sample = json.load(one_json_file)

if '논문_' in source_file:
    source_list = list(pd.DataFrame(one_json_sample['data'])['text'])

    
elif '특허_z' in source_file:
    source_df = pd.DataFrame(one_json_sample['data'])
    source_list = list(source_df[source_df['attr'] == 'abs']['text'])


elif '특허_0' in source_file or '특허_1' in source_file:
    source_df = pd.DataFrame(one_json_sample['data'])
    source_dict = dict(source_df[source_df['attr'] == '초록']['sentence'].explode())
    source_json = pd.json_normalize(source_dict)
    source_list = list(source_json.filter(regex='text').values[0])
    

elif '법령' in source_file or '판례' in source_file:
    source_df = pd.DataFrame(one_json_sample['data'])
    source_dict = dict(source_df['sentence'].explode())
    source_json = pd.json_normalize(source_dict)
    source_list = list(source_json.filter(regex='text').values[0])

In [540]:
source_list[2190]

'결과: 건강상태는 질환인식(r=-.44, p<.001), 회피대처 (r=-.33, p=.001)와 유의미한 상관관계를 나타내었다.'

In [541]:
preprocessing_sentences = formal_preprocessing_text(source_list[2190])
for preprocessing_sentence in preprocessing_sentences:
    print(preprocessing_sentence)

결과  건강상태는 질환인식(r - 44  p  001)  회피대처 (r - 33  p  001)와 유의미한 상관관계를 나타내었다.


In [542]:
print(preprocessing_sentences)

['결과  건강상태는 질환인식(r - 44  p  001)  회피대처 (r - 33  p  001)와 유의미한 상관관계를 나타내었다.']


In [553]:
train_file_name[10]

'AIHUB_전문분야 말뭉치/Training\\[원천]전문분야_train\\특허_03.json'

In [554]:
source_file = train_file_name[10]

with open(source_file, 'r', encoding='utf-8') as one_json_file:
    one_json_sample = json.load(one_json_file)

if '논문_' in source_file:
    source_list = list(pd.DataFrame(one_json_sample['data'])['text'])

    
elif '특허_z' in source_file:
    source_df = pd.DataFrame(one_json_sample['data'])
    source_list = list(source_df[source_df['attr'] == 'abs']['text'])


elif '특허_0' in source_file or '특허_1' in source_file:
    source_df = pd.DataFrame(one_json_sample['data'])
    source_dict = dict(source_df[source_df['attr'] == '초록']['sentence'].explode())
    source_json = pd.json_normalize(source_dict)
    source_list = list(source_json.filter(regex='text').values[0])
    

elif '법령' in source_file or '판례' in source_file:
    source_df = pd.DataFrame(one_json_sample['data'])
    source_dict = dict(source_df['sentence'].explode())
    source_json = pd.json_normalize(source_dict)
    source_list = list(source_json.filter(regex='text').values[0])

In [563]:
source_list[338]

'또한, [A] 화합물이 하기 화학식 (1-A)로 표시되는 화합물 및 하기 화학식 (1-B)로 표시되는 화합물로 이루어지는 군에서 선택되는 적어도 1종의 화합물인 감방사선성 조성물이다.'

In [564]:
preprocessing_sentences = formal_preprocessing_text(source_list[338])
for preprocessing_sentence in preprocessing_sentences:
    print(preprocessing_sentence)

또한   화합물이 하기 화학식 (1-A)로 표시되는 화합물 및 하기 화학식 (1-B)로 표시되는 화합물로 이루어지는 군에서 선택되는 적어도 1종의 화합물인 감방사선성 조성물이다.


In [565]:
print(preprocessing_sentences)

['또한   화합물이 하기 화학식 (1-A)로 표시되는 화합물 및 하기 화학식 (1-B)로 표시되는 화합물로 이루어지는 군에서 선택되는 적어도 1종의 화합물인 감방사선성 조성물이다.']


In [673]:
source_file = train_file_name[24]

with open(source_file, 'r', encoding='utf-8') as one_json_file:
    one_json_sample = json.load(one_json_file)

if '논문_' in source_file:
    source_list = list(pd.DataFrame(one_json_sample['data'])['text'])

    
elif '특허_z' in source_file:
    source_df = pd.DataFrame(one_json_sample['data'])
    source_list = list(source_df[source_df['attr'] == 'abs']['text'])


elif '특허_0' in source_file or '특허_1' in source_file:
    source_df = pd.DataFrame(one_json_sample['data'])
    source_dict = dict(source_df[source_df['attr'] == '초록']['sentence'].explode())
    source_json = pd.json_normalize(source_dict)
    source_list = list(source_json.filter(regex='text').values[0])
    

elif '법령' in source_file or '판례' in source_file:
    source_df = pd.DataFrame(one_json_sample['data'])
    source_dict = dict(source_df['sentence'].explode())
    source_json = pd.json_normalize(source_dict)
    source_list = list(source_json.filter(regex='text').values[0])

In [677]:
source_list[28]

'시클로헥사논 옥심 생성을 위한 암옥심화 반응기는, (a) 교반기를 구비한 반응기 용기; (b) 내부 여과 시스템; (c) 내부 액체 암모니아 증발 코일; (d) 내부 기체 암모니아 환형 분배기; (e) 외부 시클로헥사논 환형 분배기; (f) 내부 과산화수소 환형 분배기; (g) 내부 원통형 흡출관 (draft tube); 및 (h) 외부 냉각 재킷을 포함한다.'

In [676]:
preprocessing_sentences = formal_preprocessing_text(source_list[28])
for preprocessing_sentence in preprocessing_sentences:
    print(preprocessing_sentence)

시클로헥사논 옥심 생성을 위한 암옥심화 반응기는  (a) 교반기를 구비한 반응기 용기  (b) 내부 여과 시스템  (c) 내부 액체 암모니아 증발 코일  (d) 내부 기체 암모니아 환형 분배기  (e) 외부 시클로헥사논 환형 분배기  (f) 내부 과산화수소 환형 분배기  (g) 내부 원통형 흡출관 (draft tube)  및 (h) 외부 냉각 재킷을 포함한다.


In [678]:
print(preprocessing_sentences)

['시클로헥사논 옥심 생성을 위한 암옥심화 반응기는  (a) 교반기를 구비한 반응기 용기  (b) 내부 여과 시스템  (c) 내부 액체 암모니아 증발 코일  (d) 내부 기체 암모니아 환형 분배기  (e) 외부 시클로헥사논 환형 분배기  (f) 내부 과산화수소 환형 분배기  (g) 내부 원통형 흡출관 (draft tube)  및 (h) 외부 냉각 재킷을 포함한다.']


In [47]:
train_file_name[2]

'AIHUB_전문분야 말뭉치/Training\\[원천]전문분야_train\\법령_l_1000_1.json'

In [48]:
source_file = train_file_name[2]

with open(source_file, 'r', encoding='utf-8') as one_json_file:
    one_json_sample = json.load(one_json_file)

if '논문_' in source_file:
    source_list = list(pd.DataFrame(one_json_sample['data'])['text'])

    
elif '특허_z' in source_file:
    source_df = pd.DataFrame(one_json_sample['data'])
    source_list = list(source_df[source_df['attr'] == 'abs']['text'])


elif '특허_0' in source_file or '특허_1' in source_file:
    source_df = pd.DataFrame(one_json_sample['data'])
    source_dict = dict(source_df[source_df['attr'] == '초록']['sentence'].explode())
    source_json = pd.json_normalize(source_dict)
    source_list = list(source_json.filter(regex='text').values[0])
    

elif '법령' in source_file or '판례' in source_file:
    source_df = pd.DataFrame(one_json_sample['data'])
    source_dict = dict(source_df['sentence'].explode())
    source_json = pd.json_normalize(source_dict)
    source_list = list(source_json.filter(regex='text').values[0])

In [92]:
source_list[0:20]

['제1조(목적)이 영은 「정부조직법」 제12조제4항에 따라 국무회의의 운영에 필요한 사항을 규정함을 목적으로 한다. [전문개정 2011.11.7]',
 '제2조(회의운영)① 국무회의는 국가의 중요정책이 전정부적차원에서 충분히 심의될 수 있도록운영되어야 한다. ② 국무회의는 정례(정례) 국무회의와 임시 국무회의로 구분하되, 정례 국무회의는 매주1회 소집하고, 임시 국무회의는 필요에 따라 그때그때 소집한다. [전문개정 2011.11.7]',
 '제3조(의안제출)① 대통령ㆍ국무총리 또는 국무위원은 「대한민국헌법」 제89조 및 법령에 규정된 국무회의의 심의사항을 의안으로 제출한다. ② 중요 정책을 시행하기 위하여 필요한 법령안을 제출할 때에는 그 정책의 내용을 심의하기 위한 자료를 함께 제출하거나 그 정책의 내용을 심의하기 위한 의안을 미리 제출하여야 한다. ③ 국무회의에서 중점 심의되어야 할 중요 사항에 대해서는 그 심의에 필요한 검토의견 등을 해당 의안에 분명히 밝혀 제출하여야 한다. ④ 의안은 의결사항과 보고사항으로 구분하여 늦어도 그 의안을 상정할 차관회의의 개회일 3일 전까지 행정안전부에 제출하여야 한다. 다만, 법률 공포안, 「대한민국헌법」 제89조제16호에 규정된 사항에 관한 의안과 긴급한 의안은 그러하지 아니하다. 〈개정 2013ㆍ3ㆍ23, 2014ㆍ11ㆍ19, 2017ㆍ7ㆍ26〉⑤ 행정안전부장관은 차관회의의 심의를 거친 의안을 국무회의 개회일 2일전까지 의사일정과 함께 대통령, 국무총리, 국무위원 및 제8조에 따른 배석자에게 배부한다. 다만, 임시 국무회의에 의안을 상정하는 경우 또는 긴급한 의안을 상정하는 경우에는 그러하지 아니하다. 〈개정 2013ㆍ3ㆍ23, 2014ㆍ11ㆍ19, 2017ㆍ7ㆍ26〉',
 '⑥ 다음 각 호의 어느 하나에 해당하는 사항으로서 중요 정책에 관계되는 사항은 국무회의에 수시로 보고하여야 한다. 〈개정 2017ㆍ7ㆍ26〉1. 국내외 중요 정보의 분석 상황2. 정부의 역점사업 추진현황3. 국민생활에 영향을 미치는 중요시책의 

In [93]:
source_list[12]

'제1조(설치 및 기능)① 행정 각 부ㆍ처ㆍ청 간의 협조를 긴밀하게 하며 국무회의에 제출된 의안과 국무회의로부터 지시받은 사항을 심의하기 위하여 차관회의를 둔다. ② 차관회의는 국무에 관하여 국무회의에 건의할 수 있다. [전문개정 2011.11.7]'

In [94]:
print(source_list[116], end="\n\n")
preprocessing_sentences = formal_preprocessing_text(source_list[12])
for preprocessing_sentence in preprocessing_sentences:
    print(preprocessing_sentence)

제9조(심문과 진술권)①징계위원회는 제8조제1항에 따라 출석한 징계등 혐의자에게 혐의내용에 관한 심문을 하고 필요하다고 인정할 때에는 관계인의 출석을 요구해 심문할 수 있다. 〈개정 1978. 5. 27., 2019. 2. 26.〉② 징계위원회는 징계등 혐의자에게 충분한 진술을 할 수 있는 기회를 부여해야 하며, 징계등 혐의자는 별지 제3호의2서식의 의견서 또는 구술로 자기에게 이익이 되는 사실을 진술하며 증거를 제출할 수 있다. 〈개정 2019ㆍ2ㆍ26, 2019ㆍ8ㆍ6〉③ 징계등 혐의자는 증인의 심문을 신청할 수 있다. 이 경우 징계위원회는 증인 채택 여부를 결정해야 한다. 〈개정 2019ㆍ2ㆍ26〉④ 징계등 의결 요구자 및 징계등 의결 요구의 신청자는 징계위원회에 출석하여 의견을 진술하거나 서면으로 의견을 진술할 수 있다. 다만, 중징계 또는 중징계 관련 징계부가금 요구사건의 경우에는 특별한 사유가 없으면 징계위원회에 출석하여 의견을 진술해야 한다. 〈개정 1994. 6. 24., 2019. 2. 26., 2020. 7. 28.〉⑤ 징계등 의결 요구자 및 징계등 의결 요구의 신청자는 「감사원법」 제32조제1항 및 제10항에 따라 감사원이 파면, 해임, 강등 또는 정직 중 어느 하나의 징계처분을 요구한 사건에 대해서는 징계위원회개최 일시ㆍ장소 등을 감사원에 통보해야 한다. 〈개정 2019ㆍ2ㆍ26〉⑥ 감사원은 제5항에 따른 통보를 받은 경우 소속 공무원의 징계위원회 출석을 관할 징계위원회에 요청할 수 있으며, 관할 징계위원회는 출석 허용 여부를 결정해야 한다. 〈신설 2019ㆍ2ㆍ26〉

제1조(설치 및 기능)  행정 각 부 처 청 간의 협조를 긴밀하게 하며 국무회의에 제출된 의안과 국무회의로부터 지시받은 사항을 심의하기 위하여 차관회의를 둔다.


In [87]:
print(source_list[153], end="\n\n")
preprocessing_sentences = formal_preprocessing_text(source_list[153])
for preprocessing_sentence in preprocessing_sentences:
    print(preprocessing_sentence)

제16조(의결정족수)각 위원회의 의사는 위원장·부위원장을 포함한 재적위원 과반수의 출석과 출석위원과반수의 찬성으로 의결한다. 다만, 자격인정의 결정에 있어서는 출석위원 3분의2이상의 찬성으로 행한다. 〈개정 78·12·30, 2000·7·10〉

제16조(의결정족수)각 위원회의 의사는 위원장 부위원장을 포함한 재적위원 과반수의 출석과 출석위원과반수의 찬성으로 의결한다.


In [633]:
train_file_name[30]

'AIHUB_전문분야 말뭉치/Training\\[원천]전문분야_train\\판례_case03.json'

In [11]:
source_file = train_file_name[30]

with open(source_file, 'r', encoding='utf-8') as one_json_file:
    one_json_sample = json.load(one_json_file)

if '논문_' in source_file:
    source_list = list(pd.DataFrame(one_json_sample['data'])['text'])

    
elif '특허_z' in source_file:
    source_df = pd.DataFrame(one_json_sample['data'])
    source_list = list(source_df[source_df['attr'] == 'abs']['text'])


elif '특허_0' in source_file or '특허_1' in source_file:
    source_df = pd.DataFrame(one_json_sample['data'])
    source_dict = dict(source_df[source_df['attr'] == '초록']['sentence'].explode())
    source_json = pd.json_normalize(source_dict)
    source_list = list(source_json.filter(regex='text').values[0])
    

elif '법령' in source_file or '판례' in source_file:
    source_df = pd.DataFrame(one_json_sample['data'])
    source_dict = dict(source_df['sentence'].explode())
    source_json = pd.json_normalize(source_dict)
    source_list = list(source_json.filter(regex='text').values[0])

In [17]:
source_list[92]

'제93조(탈법방법에 의한 문서ㆍ도화의 배부ㆍ게시 등 금지) ① 누구든지 선거일전 180일(보궐선거 등에 있어서는 그 선거의 실시사유가 확정된 때)부터 선거일까지 선거에 영향을 미치게 하기 위하여 이 법의 규정에 의하지 아니하고는 정당(창당준비위원회와 정당의 정강ㆍ정책을 포함한다.'

In [43]:
source_list[400:420]

['이 경우에 교육학ㆍ윤리학등 학교장이 특히 인정하는 학과목의 교육경력 연수는 이를 연구실적 연수로 대체할 수 있다.',
 '3.국내의학교교육으로는이수할수없었거나 곤란하였던 학과목을 담당할 자로서 연구실적이 별표의 연구실적 연수에 해당하거나, 논문ㆍ저서 기타 연구업적이 현저하나 학력이 별표의 기준에 해당하지 아니한 자',
 '4.국가공무원법의 규정에 의하여 자연과학계의 연구관 또는 5급 이상의 농촌지도직공무원으로 근무하고 있는 자',
 '공무원수당 등에 관한 규정(2013. 3. 23. 대통령령 제24425호로 개정된 것)',
 '제7조의2(성과상여금) ① 소속 장관은 별표 2의2에 따른 공무원 중 근무성적, 업무실적 등이 우수한 사람에게는 예산의 범위에서 성과상여금을 지급한다.',
 '국공립대학연구보조비 지급규정(2010. 9. 6. 교육과학기술부훈령 제190호로 개정된 것)',
 '제2조(교부) ① 연구보조비는 대학, 산업대학, 교육대학, 전문대학, 방송대학ㆍ통신대학 및 방송통신대학(이하 "대상학교"라 한다)의 기성회 회장에게 교부한다.',
 '제3조(지급) ① 연구보조비는 지급대상학교 교원(재외한국인학자 유치계획에 의하여 유치한 교원을 포함한다)과 조교에게 기성회 회장이 예산의 범위 안에서 지급한다.',
 '② 제1항의 연구보조비는 교원의 교육ㆍ연구 및 봉사활동 등의 실적과 조교의 근무실적 등 대상학교에서 정한 평가방법에 따라 지급하는 성과급(이하 "성과급"이라 한다)과 시간강사의 강의시수에 따라 지급하는 시간강사연구보조비(이하 "시간강사연구보조비"라 한다)로 구분하여 지급하며 그 지급기준은 교육과학기술부장관이 정한다.',
 '이 경우 성과급 지급액은 교육과학기술부장관이 정하는 지급기준과 대상학교에 교부되는 성과급연구비 총액의 범위 안에서 학교의 장이 정한다.',
 '④ 성과급의 지급액은 대상학교에 교부되는 성과급 연구비 총액의 범위 안에서 대상학교별 연구보조비 지급심의위원회의 심의를 거쳐 학교의 장이 정한다.',
 '⑤ 성과급은 연4회 이내로 지급하고,

In [46]:
source_list[406]

'제2조(교부) ① 연구보조비는 대학, 산업대학, 교육대학, 전문대학, 방송대학ㆍ통신대학 및 방송통신대학(이하 "대상학교"라 한다)의 기성회 회장에게 교부한다.'

In [44]:
preprocessing_sentences = formal_preprocessing_text(source_list[406])
for preprocessing_sentence in preprocessing_sentences:
    print(preprocessing_sentence)

제2조(교부)   연구보조비는 대학  산업대학  교육대학  전문대학  방송대학 통신대학 및 방송통신대학(이하  대상학교 라 한다)의 기성회 회장에게 교부한다.


In [45]:
print(preprocessing_sentences)

['제2조(교부)   연구보조비는 대학  산업대학  교육대학  전문대학  방송대학 통신대학 및 방송통신대학(이하  대상학교 라 한다)의 기성회 회장에게 교부한다.']


### AIHUB 대규모 웹데이터 기반 한국어 말뭉치 데이터

In [485]:
path_list = ['AIHUB_웹데이터 기반 한국어 말뭉치 데이터/Training/원천데이터/TS1/IT_과학/' + '**/*.json',
'AIHUB_웹데이터 기반 한국어 말뭉치 데이터/Training/원천데이터/TS1/건강/' + '**/*.json',
'AIHUB_웹데이터 기반 한국어 말뭉치 데이터/Training/원천데이터/TS1/경제/' + '**/*.json',
'AIHUB_웹데이터 기반 한국어 말뭉치 데이터/Training/원천데이터/TS1/교육/' + '**/*.json',
'AIHUB_웹데이터 기반 한국어 말뭉치 데이터/Training/원천데이터/TS1/국제/' + '**/*.json',
'AIHUB_웹데이터 기반 한국어 말뭉치 데이터/Training/원천데이터/TS1/라이프스타일/' + '**/*.json',
'AIHUB_웹데이터 기반 한국어 말뭉치 데이터/Training/원천데이터/TS1/문화/' + '**/*.json',
'AIHUB_웹데이터 기반 한국어 말뭉치 데이터/Training/원천데이터/TS1/사건사고/' + '**/*.json',
'AIHUB_웹데이터 기반 한국어 말뭉치 데이터/Training/원천데이터/TS1/사회일반/' + '**/*.json',
'AIHUB_웹데이터 기반 한국어 말뭉치 데이터/Training/원천데이터/TS1/산업/' + '**/*.json',
'AIHUB_웹데이터 기반 한국어 말뭉치 데이터/Training/원천데이터/TS1/스포츠/' + '**/*.json',
'AIHUB_웹데이터 기반 한국어 말뭉치 데이터/Training/원천데이터/TS1/여성복지/' + '**/*.json',
'AIHUB_웹데이터 기반 한국어 말뭉치 데이터/Training/원천데이터/TS1/여행레저/' + '**/*.json',
'AIHUB_웹데이터 기반 한국어 말뭉치 데이터/Training/원천데이터/TS1/연예/' + '**/*.json',
'AIHUB_웹데이터 기반 한국어 말뭉치 데이터/Training/원천데이터/TS1/정치/' + '**/*.json',
'AIHUB_웹데이터 기반 한국어 말뭉치 데이터/Training/원천데이터/TS1/지역/' + '**/*.json',
'AIHUB_웹데이터 기반 한국어 말뭉치 데이터/Training/원천데이터/TS1/취미/' + '**/*.json',
'AIHUB_웹데이터 기반 한국어 말뭉치 데이터/Validation/원천데이터/VS1/IT_과학/' + '**/*.json',
'AIHUB_웹데이터 기반 한국어 말뭉치 데이터/Validation/원천데이터/VS1/건강/' + '**/*.json',
'AIHUB_웹데이터 기반 한국어 말뭉치 데이터/Validation/원천데이터/VS1/경제/' + '**/*.json',
'AIHUB_웹데이터 기반 한국어 말뭉치 데이터/Validation/원천데이터/VS1/교육/' + '**/*.json',
'AIHUB_웹데이터 기반 한국어 말뭉치 데이터/Validation/원천데이터/VS1/국제/' + '**/*.json',
'AIHUB_웹데이터 기반 한국어 말뭉치 데이터/Validation/원천데이터/VS1/라이프스타일/' + '**/*.json',
'AIHUB_웹데이터 기반 한국어 말뭉치 데이터/Validation/원천데이터/VS1/문화/' + '**/*.json',
'AIHUB_웹데이터 기반 한국어 말뭉치 데이터/Validation/원천데이터/VS1/사건사고/' + '**/*.json',
'AIHUB_웹데이터 기반 한국어 말뭉치 데이터/Validation/원천데이터/VS1/사회일반/' + '**/*.json',
'AIHUB_웹데이터 기반 한국어 말뭉치 데이터/Validation/원천데이터/VS1/산업/' + '**/*.json',
'AIHUB_웹데이터 기반 한국어 말뭉치 데이터/Validation/원천데이터/VS1/스포츠/' + '**/*.json',
'AIHUB_웹데이터 기반 한국어 말뭉치 데이터/Validation/원천데이터/VS1/여성복지/' + '**/*.json',
'AIHUB_웹데이터 기반 한국어 말뭉치 데이터/Validation/원천데이터/VS1/여행레저/' + '**/*.json',
'AIHUB_웹데이터 기반 한국어 말뭉치 데이터/Validation/원천데이터/VS1/연예/' + '**/*.json',
'AIHUB_웹데이터 기반 한국어 말뭉치 데이터/Validation/원천데이터/VS1/정치/' + '**/*.json',
'AIHUB_웹데이터 기반 한국어 말뭉치 데이터/Validation/원천데이터/VS1/지역/' + '**/*.json',
'AIHUB_웹데이터 기반 한국어 말뭉치 데이터/Validation/원천데이터/VS1/취미/' + '**/*.json',]

train_file_name_01 = glob(path_list[0], recursive = True)
train_file_name_02 = glob(path_list[1], recursive = True)
train_file_name_03 = glob(path_list[2], recursive = True)
train_file_name_04 = glob(path_list[3], recursive = True)
train_file_name_05 = glob(path_list[4], recursive = True)
train_file_name_06 = glob(path_list[5], recursive = True)
train_file_name_07 = glob(path_list[6], recursive = True)
train_file_name_08 = glob(path_list[7], recursive = True)
train_file_name_09 = glob(path_list[8], recursive = True)
train_file_name_10 = glob(path_list[9], recursive = True)
train_file_name_11 = glob(path_list[10], recursive = True)
train_file_name_12 = glob(path_list[11], recursive = True)
train_file_name_13 = glob(path_list[12], recursive = True)
train_file_name_14 = glob(path_list[13], recursive = True)
train_file_name_15 = glob(path_list[14], recursive = True)
train_file_name_16 = glob(path_list[15], recursive = True)
train_file_name_17 = glob(path_list[16], recursive = True)

valid_file_name_01 = glob(path_list[17], recursive = True)
valid_file_name_02 = glob(path_list[18], recursive = True)
valid_file_name_03 = glob(path_list[19], recursive = True)
valid_file_name_04 = glob(path_list[20], recursive = True)
valid_file_name_05 = glob(path_list[21], recursive = True)
valid_file_name_06 = glob(path_list[22], recursive = True)
valid_file_name_07 = glob(path_list[23], recursive = True)
valid_file_name_08 = glob(path_list[24], recursive = True)
valid_file_name_09 = glob(path_list[25], recursive = True)
valid_file_name_10 = glob(path_list[26], recursive = True)
valid_file_name_11 = glob(path_list[27], recursive = True)
valid_file_name_12 = glob(path_list[28], recursive = True)
valid_file_name_13 = glob(path_list[29], recursive = True)
valid_file_name_14 = glob(path_list[30], recursive = True)
valid_file_name_15 = glob(path_list[31], recursive = True)
valid_file_name_16 = glob(path_list[32], recursive = True)
valid_file_name_17 = glob(path_list[33], recursive = True)

In [486]:
with open(train_file_name_13[0], 'r', encoding='utf-8') as one_json_file:
    one_json_sample = json.load(one_json_file)

source_list = list(pd.DataFrame(one_json_sample['SJML']['text'])['content'])

In [679]:
train_file_name_13[0]

'AIHUB_웹데이터 기반 한국어 말뭉치 데이터/Training/원천데이터/TS1/여행레저\\BWTL217000058479.json'

In [496]:
source_list[6]

'. 행정자치부가 자전거 주간을 맞아 23일 머니위크가 주최한 청계릴레이사이클링(청계천 가족자전거 운동회)를 공식후원하고 현장에서 자전거 안전 캠페인을 실시했다.. . 자전거 정책을 맡은 행정자치부 주민생활환경과는 청계천 한빛광장에 마련된 무대에서 운동회 참가자와 주말 청계천 나들이객을 대상으로 자전거 안전운전요령 등 홍보책자, 전조등 등 기념품 등을 배부하고 안전한 자전거타기 홍보 캠페인을 실시했다.. . 이날 행사에서 허만영 행정자치부 주민생활환경과장은 “최근 자전거가 1000만대 수준으로 크게 늘면서 자전거사고도 매년 10%씩 증가하고 있다”면서 “이제는 안전하게 자전거를 타는 것이 무엇보다도 중요하다”고 강조하며 행사에 참여한 시민들에게 자전거 안전문화 확산에 적극 동참해줄 것을 당부했다.. . 한편 행정자치부는 2016년도 자전거도로정비 사업비 20억원을 지자체에 지원해 생활형 자전거이용이 많은 도심지 자전거·보행자겸용도로정비 개선사업을 추진해 안전하고 편리한 자전거 (이름)경을 (이름)다.. . 또한 자전거사고 취약계층인 어린이와 어르신들의 자전거 안전사고 예방을 위해 자전거 민간단체와 협업을 통해 자전거 안전교실을 운영(5월~11월)한다.'

In [493]:
pprint(source_list[6])

('. 행정자치부가 자전거 주간을 맞아 23일 머니위크가 주최한 청계릴레이사이클링(청계천 가족자전거 운동회)를 공식후원하고 현장에서 자전거 '
 '안전 캠페인을 실시했다.. . 자전거 정책을 맡은 행정자치부 주민생활환경과는 청계천 한빛광장에 마련된 무대에서 운동회 참가자와 주말 '
 '청계천 나들이객을 대상으로 자전거 안전운전요령 등 홍보책자, 전조등 등 기념품 등을 배부하고 안전한 자전거타기 홍보 캠페인을 실시했다.. '
 '. 이날 행사에서 허만영 행정자치부 주민생활환경과장은 “최근 자전거가 1000만대 수준으로 크게 늘면서 자전거사고도 매년 10%씩 '
 '증가하고 있다”면서 “이제는 안전하게 자전거를 타는 것이 무엇보다도 중요하다”고 강조하며 행사에 참여한 시민들에게 자전거 안전문화 확산에 '
 '적극 동참해줄 것을 당부했다.. . 한편 행정자치부는 2016년도 자전거도로정비 사업비 20억원을 지자체에 지원해 생활형 자전거이용이 '
 '많은 도심지 자전거·보행자겸용도로정비 개선사업을 추진해 안전하고 편리한 자전거 (이름)경을 (이름)다.. . 또한 자전거사고 취약계층인 '
 '어린이와 어르신들의 자전거 안전사고 예방을 위해 자전거 민간단체와 협업을 통해 자전거 안전교실을 운영(5월~11월)한다.')


In [494]:
preprocessing_sentences = formal_preprocessing_text(source_list[6])
for preprocessing_sentence in preprocessing_sentences:
    print(preprocessing_sentence)

자전거 정책을 맡은 행정자치부 주민생활환경과는 청계천 한빛광장에 마련된 무대에서 운동회 참가자와 주말 청계천 나들이객을 대상으로 자전거 안전운전요령 등 홍보책자  전조등 등 기념품 등을 배부하고 안전한 자전거타기 홍보 캠페인을 실시했다.
이날 행사에서 허만영 행정자치부 주민생활환경과장은  최근 자전거가 1000만대 수준으로 크게 늘면서 자전거사고도 매년 10 씩 증가하고 있다 면서  이제는 안전하게 자전거를 타는 것이 무엇보다도 중요하다 고 강조하며 행사에 참여한 시민들에게 자전거 안전문화 확산에 적극 동참해줄 것을 당부했다.
한편 행정자치부는 2016년도 자전거도로정비 사업비 20억원을 지자체에 지원해 생활형 자전거이용이 많은 도심지 자전거 보행자겸용도로정비 개선사업을 추진해 안전하고 편리한 자전거 (이름)경을 (이름)다.
또한 자전거사고 취약계층인 어린이와 어르신들의 자전거 안전사고 예방을 위해 자전거 민간단체와 협업을 통해 자전거 안전교실을 운영(5월 11월)한다.


In [495]:
print(preprocessing_sentences)

['자전거 정책을 맡은 행정자치부 주민생활환경과는 청계천 한빛광장에 마련된 무대에서 운동회 참가자와 주말 청계천 나들이객을 대상으로 자전거 안전운전요령 등 홍보책자  전조등 등 기념품 등을 배부하고 안전한 자전거타기 홍보 캠페인을 실시했다.', '이날 행사에서 허만영 행정자치부 주민생활환경과장은  최근 자전거가 1000만대 수준으로 크게 늘면서 자전거사고도 매년 10 씩 증가하고 있다 면서  이제는 안전하게 자전거를 타는 것이 무엇보다도 중요하다 고 강조하며 행사에 참여한 시민들에게 자전거 안전문화 확산에 적극 동참해줄 것을 당부했다.', '한편 행정자치부는 2016년도 자전거도로정비 사업비 20억원을 지자체에 지원해 생활형 자전거이용이 많은 도심지 자전거 보행자겸용도로정비 개선사업을 추진해 안전하고 편리한 자전거 (이름)경을 (이름)다.', '또한 자전거사고 취약계층인 어린이와 어르신들의 자전거 안전사고 예방을 위해 자전거 민간단체와 협업을 통해 자전거 안전교실을 운영(5월 11월)한다.']


### AIHUB 산업정보 연계 주요국 특허 영-한 데이터

In [681]:
path_list = ['AIHUB_산업정보 연계 주요국 특허 영-한 데이터/Training/'+ '/**/*.json', 
             'AIHUB_산업정보 연계 주요국 특허 영-한 데이터/Validation/'+ '/**/*.json']
train_file_name, valid_file_name = train_valid_json_file_name_list(path_list)

In [481]:
with open(train_file_name[130], 'r', encoding='utf-8') as one_json_file:
    one_json_sample = json.load(one_json_file)
    
try: 
    source_list = list(pd.DataFrame(one_json_sample['labeled_data'])['astrt_cont_kor'])

except:
    pass   

In [682]:
train_file_name[130]

'AIHUB_산업정보 연계 주요국 특허 영-한 데이터/Training\\라벨링데이터_0831_add\\TL_2\\기타 유리제품 제조업.json'

In [482]:
source_list[79]

'본 발명은, 공정 단계 이산화규소 과립을 제공하는 단계, 용융 도가니에서 상기 이산화규소 과립으로부터 유리 용융물을 제조하는 단계, 상기 유리 용융물의 적어도 일부로부터 석영 유리체를 제조하는 단계, ⅳ.) 상기 석영 유리체를 화학적, 열적, 또는 기계적 처리로 이루어진 군으로부터 선택된 적어도 하나의 절차로 처리하여, 처리된 석영 유리체를 얻는, 처리 단계를 포함하는, 석영 유리체의 제조 공정에 관한 것이다. 본 발명은 또한 상기 공정에 의해 얻을 수 있는 석영 유리체에 관한 것이다. 본 발명은 또한, 상기 석영 유리체의 추가 가공 단계에 의해 각각 얻을 수 있는, 광 가이드, 발광체 및 성형체와 관련된다.'

In [483]:
preprocessing_sentences = formal_preprocessing_text(source_list[79])
for preprocessing_sentence in preprocessing_sentences:
    print(preprocessing_sentence)

본 발명은  공정 단계 이산화규소 과립을 제공하는 단계  용융 도가니에서 상기 이산화규소 과립으로부터 유리 용융물을 제조하는 단계  상기 유리 용융물의 적어도 일부로부터 석영 유리체를 제조하는 단계    ) 상기 석영 유리체를 화학적  열적  또는 기계적 처리로 이루어진 군으로부터 선택된 적어도 하나의 절차로 처리하여  처리된 석영 유리체를 얻는  처리 단계를 포함하는  석영 유리체의 제조 공정에 관한 것이다.
본 발명은 또한 상기 공정에 의해 얻을 수 있는 석영 유리체에 관한 것이다.
본 발명은 또한  상기 석영 유리체의 추가 가공 단계에 의해 각각 얻을 수 있는  광 가이드  발광체 및 성형체와 관련된다.


In [484]:
print(preprocessing_sentences)

['본 발명은  공정 단계 이산화규소 과립을 제공하는 단계  용융 도가니에서 상기 이산화규소 과립으로부터 유리 용융물을 제조하는 단계  상기 유리 용융물의 적어도 일부로부터 석영 유리체를 제조하는 단계    ) 상기 석영 유리체를 화학적  열적  또는 기계적 처리로 이루어진 군으로부터 선택된 적어도 하나의 절차로 처리하여  처리된 석영 유리체를 얻는  처리 단계를 포함하는  석영 유리체의 제조 공정에 관한 것이다.', '본 발명은 또한 상기 공정에 의해 얻을 수 있는 석영 유리체에 관한 것이다.', '본 발명은 또한  상기 석영 유리체의 추가 가공 단계에 의해 각각 얻을 수 있는  광 가이드  발광체 및 성형체와 관련된다.']


### AIHUB 논문자료 요약

In [416]:
path_list = ['AIHUB_논문자료 요약/Training/'+ '/**/*.json', 
             'AIHUB_논문자료 요약/Validation/'+ '/**/*.json']
train_file_name, valid_file_name = train_valid_json_file_name_list(path_list)

In [417]:
train_file_name[0]

'AIHUB_논문자료 요약/Training\\training_논문\\논문요약_0206_0.json'

In [422]:
source_file = train_file_name[0]
with open(source_file, 'r', encoding='utf-8') as one_json_file:
    one_json_sample = json.load(one_json_file)

source_df = pd.DataFrame(one_json_sample['data'])

if '논문/논문요약' or '특허섹션' in source_file:    
    summary_entire_source_dict = dict(source_df['summary_entire'].explode())
    summary_entire_source_json = pd.json_normalize(summary_entire_source_dict)
    summary_entire_source_list = list(summary_entire_source_json.filter(regex='orginal_text').values[0])

    summary_section_source_dict = dict(source_df['summary_section'].explode())
    summary_section_source_json = pd.json_normalize(summary_section_source_dict)
    summary_section_source_list = list(summary_section_source_json.filter(regex='orginal_text').values[0])

else:
    summary_section_source_dict = dict(source_df['summary_section'].explode())
    summary_section_source_json = pd.json_normalize(summary_section_source_dict)
    summary_section_source_list = list(summary_section_source_json.filter(regex='orginal_text').values[0])

In [453]:
summary_entire_source_list[10]

'사회불안과 섭식문제는 공병률이 높은 병리인데, 본 연구는 전자가 후자로 옮아가는 과정을 인지적 변인인 이분법 사고와 반추가 순차적으로 매개하는지를 알아보고자 했다. 이를 위해 성인 여성 187명에게 한국판 식사태도 검사(KEAT-26), 사회공포증 척도(SPS), 이분법적 사고 지표 확장판(DTI-35), 반추적 사고 스타일 질문지(RTSQ)를 온라인 설문으로 실시했다. SPSS 24.0과 Process Macro 2.16을 사용해서 이분법 사고와 반추 각각을 매개 변인으로 하는 두 개의 단순 매개 분석을 했고, 이어서 두 변인이 한 모형 안에서 차례대로 작용하여 사회불안과 섭식문제의 관계를 직렬 매개하는지를 분석했다. 먼저, 단순매개분석에서 이분법 사고는 사회불안과 이상섭식행동의 관계를 부분 매개하여서, 사회불안이 높으면 사소한 실수도 파국화하는 이분법적 사고로 인해 이상섭식행동이 야기될 수 있음이 시사되었다. 이어서, 반추적 사고도 이들의 관계를 부분 매개하여서 사회불안이 높은 사람들이 사후처리로 반추적 사고를 한 결과로 이상섭식행동이 증가될 수 있음이 시사되었다. 최종적인 직렬매개분석에서는 두 매개 변인의 순차적 매개 효과가 유의하여서, 사회불안이 높은 경우 이상에 비춰 현실을 이분법적으로 판단하면 그 괴리로 반추적 사고가 촉발되어 이상섭식행동으로 이어질 수 있음이 시사되었다. 끝으로 연구의 한계점과 향후 연구를 위한 제언을 제시하였다.'

In [454]:
preprocessing_sentences = formal_preprocessing_text(summary_entire_source_list[10])
for preprocessing_sentence in preprocessing_sentences:
    print(preprocessing_sentence)

사회불안과 섭식문제는 공병률이 높은 병리인데  본 연구는 전자가 후자로 옮아가는 과정을 인지적 변인인 이분법 사고와 반추가 순차적으로 매개하는지를 알아보고자 했다.
이를 위해 성인 여성 187명에게 한국판 식사태도 검사(KEAT-26)  사회공포증 척도(SPS)  이분법적 사고 지표 확장판(DTI-35)  반추적 사고 스타일 질문지(RTSQ)를 온라인 설문으로 실시했다.
SPSS 24 0과 Process Macro 2 16을 사용해서 이분법 사고와 반추 각각을 매개 변인으로 하는 두 개의 단순 매개 분석을 했고  이어서 두 변인이 한 모형 안에서 차례대로 작용하여 사회불안과 섭식문제의 관계를 직렬 매개하는지를 분석했다.
먼저  단순매개분석에서 이분법 사고는 사회불안과 이상섭식행동의 관계를 부분 매개하여서  사회불안이 높으면 사소한 실수도 파국화하는 이분법적 사고로 인해 이상섭식행동이 야기될 수 있음이 시사되었다.
이어서  반추적 사고도 이들의 관계를 부분 매개하여서 사회불안이 높은 사람들이 사후처리로 반추적 사고를 한 결과로 이상섭식행동이 증가될 수 있음이 시사되었다.
최종적인 직렬매개분석에서는 두 매개 변인의 순차적 매개 효과가 유의하여서  사회불안이 높은 경우 이상에 비춰 현실을 이분법적으로 판단하면 그 괴리로 반추적 사고가 촉발되어 이상섭식행동으로 이어질 수 있음이 시사되었다.
끝으로 연구의 한계점과 향후 연구를 위한 제언을 제시하였다.


In [455]:
print(preprocessing_sentences)

['사회불안과 섭식문제는 공병률이 높은 병리인데  본 연구는 전자가 후자로 옮아가는 과정을 인지적 변인인 이분법 사고와 반추가 순차적으로 매개하는지를 알아보고자 했다.', '이를 위해 성인 여성 187명에게 한국판 식사태도 검사(KEAT-26)  사회공포증 척도(SPS)  이분법적 사고 지표 확장판(DTI-35)  반추적 사고 스타일 질문지(RTSQ)를 온라인 설문으로 실시했다.', 'SPSS 24 0과 Process Macro 2 16을 사용해서 이분법 사고와 반추 각각을 매개 변인으로 하는 두 개의 단순 매개 분석을 했고  이어서 두 변인이 한 모형 안에서 차례대로 작용하여 사회불안과 섭식문제의 관계를 직렬 매개하는지를 분석했다.', '먼저  단순매개분석에서 이분법 사고는 사회불안과 이상섭식행동의 관계를 부분 매개하여서  사회불안이 높으면 사소한 실수도 파국화하는 이분법적 사고로 인해 이상섭식행동이 야기될 수 있음이 시사되었다.', '이어서  반추적 사고도 이들의 관계를 부분 매개하여서 사회불안이 높은 사람들이 사후처리로 반추적 사고를 한 결과로 이상섭식행동이 증가될 수 있음이 시사되었다.', '최종적인 직렬매개분석에서는 두 매개 변인의 순차적 매개 효과가 유의하여서  사회불안이 높은 경우 이상에 비춰 현실을 이분법적으로 판단하면 그 괴리로 반추적 사고가 촉발되어 이상섭식행동으로 이어질 수 있음이 시사되었다.', '끝으로 연구의 한계점과 향후 연구를 위한 제언을 제시하였다.']


In [447]:
summary_section_source_list[10]

'이상섭식행동(abnormal eating behavior)은 체중 및 체형에 대한 염려로 음식 섭취를 지나치게 제한하거나, 식욕을 억제하다가 참지 못하고 짧은 시간 동안 많은 양의 음식을 먹어치우는 등 정상범위를 벗어난 섭식행동을 가리킨다. 우울장애, 양극성장애 등과 공병률이 높기는 하지만, 섭식 삽화와 관련된 이상섭식행동을 야기하는 것은 우울보다는 불안에 더 가까워 보인다. 이상섭식행동과 연속선상에 있는 섭식장애에서는 우울할 때보다 불안할 때 거식이나 폭식 행동을 더 많이 하며, 불안장애가 선행되는 경우가 많아서 불안장애 또는 사회불안장애의 진단 등이 섭식장애의 기질적 위험 요인으로 제시되기도 한다(American Psychiatric Association, 2013/2015). 섭식장애에 동반되는 불안장애 중에서 가장 흔한 것은 사회불안장애이며, 마찬가지로 섭식장에에 선행하는 경향을 나타낸다. 높은 공병률과 발생 양상에 더해 사회불안이 섭식장애의 치료를 방해하는 요인으로 작용한다는 것이 밝혀지면서 두 병리를 연결하는 메커니즘을 탐색하는 연구들이 이어졌다. 둘을 매개하는 변인으로 부정적 평가의 두려움, 정서 표현억제, 완벽주의, 자존감과 스트레스 반응성 등이 제안되었으나, 그 연결 메커니즘은 아직까지 분명하게 밝혀지지 않았다. 국내에서는 사회불안과 폭식행동의 관계에서 인지왜곡의 역할을 탐색한 연구를 제외하면, 사회불안과 이상섭식행동의 관계를 다룬 경우가 거의 없어서 연구의 필요성이 제기된다.'

In [448]:
preprocessing_sentences = formal_preprocessing_text(summary_section_source_list[10])
for preprocessing_sentence in preprocessing_sentences:
    print(preprocessing_sentence)

이상섭식행동(abnormal eating behavior)은 체중 및 체형에 대한 염려로 음식 섭취를 지나치게 제한하거나  식욕을 억제하다가 참지 못하고 짧은 시간 동안 많은 양의 음식을 먹어치우는 등 정상범위를 벗어난 섭식행동을 가리킨다.
우울장애  양극성장애 등과 공병률이 높기는 하지만  섭식 삽화와 관련된 이상섭식행동을 야기하는 것은 우울보다는 불안에 더 가까워 보인다.
이상섭식행동과 연속선상에 있는 섭식장애에서는 우울할 때보다 불안할 때 거식이나 폭식 행동을 더 많이 하며  불안장애가 선행되는 경우가 많아서 불안장애 또는 사회불안장애의 진단 등이 섭식장애의 기질적 위험 요인으로 제시되기도 한다(American Psychiatric Association  2013 2015)  섭식장애에 동반되는 불안장애 중에서 가장 흔한 것은 사회불안장애이며  마찬가지로 섭식장에에 선행하는 경향을 나타낸다.
높은 공병률과 발생 양상에 더해 사회불안이 섭식장애의 치료를 방해하는 요인으로 작용한다는 것이 밝혀지면서 두 병리를 연결하는 메커니즘을 탐색하는 연구들이 이어졌다.
둘을 매개하는 변인으로 부정적 평가의 두려움  정서 표현억제  완벽주의  자존감과 스트레스 반응성 등이 제안되었으나  그 연결 메커니즘은 아직까지 분명하게 밝혀지지 않았다.
국내에서는 사회불안과 폭식행동의 관계에서 인지왜곡의 역할을 탐색한 연구를 제외하면  사회불안과 이상섭식행동의 관계를 다룬 경우가 거의 없어서 연구의 필요성이 제기된다.


In [449]:
print(preprocessing_sentences)

['이상섭식행동(abnormal eating behavior)은 체중 및 체형에 대한 염려로 음식 섭취를 지나치게 제한하거나  식욕을 억제하다가 참지 못하고 짧은 시간 동안 많은 양의 음식을 먹어치우는 등 정상범위를 벗어난 섭식행동을 가리킨다.', '우울장애  양극성장애 등과 공병률이 높기는 하지만  섭식 삽화와 관련된 이상섭식행동을 야기하는 것은 우울보다는 불안에 더 가까워 보인다.', '이상섭식행동과 연속선상에 있는 섭식장애에서는 우울할 때보다 불안할 때 거식이나 폭식 행동을 더 많이 하며  불안장애가 선행되는 경우가 많아서 불안장애 또는 사회불안장애의 진단 등이 섭식장애의 기질적 위험 요인으로 제시되기도 한다(American Psychiatric Association  2013 2015)  섭식장애에 동반되는 불안장애 중에서 가장 흔한 것은 사회불안장애이며  마찬가지로 섭식장에에 선행하는 경향을 나타낸다.', '높은 공병률과 발생 양상에 더해 사회불안이 섭식장애의 치료를 방해하는 요인으로 작용한다는 것이 밝혀지면서 두 병리를 연결하는 메커니즘을 탐색하는 연구들이 이어졌다.', '둘을 매개하는 변인으로 부정적 평가의 두려움  정서 표현억제  완벽주의  자존감과 스트레스 반응성 등이 제안되었으나  그 연결 메커니즘은 아직까지 분명하게 밝혀지지 않았다.', '국내에서는 사회불안과 폭식행동의 관계에서 인지왜곡의 역할을 탐색한 연구를 제외하면  사회불안과 이상섭식행동의 관계를 다룬 경우가 거의 없어서 연구의 필요성이 제기된다.']


### AIHUB 요약문 및 레포트 생성 데이터

In [127]:
path_list = ['AIHUB_요약문 및 레포트 생성 데이터/Training/원천데이터/TS1/01.news_r/' + '**/*.json',
'AIHUB_요약문 및 레포트 생성 데이터/Training/원천데이터/TS1/02.briefing/' + '**/*.json',
'AIHUB_요약문 및 레포트 생성 데이터/Training/원천데이터/TS1/03.his_cul/' + '**/*.json',
'AIHUB_요약문 및 레포트 생성 데이터/Training/원천데이터/TS1/04.paper/' + '**/*.json',
'AIHUB_요약문 및 레포트 생성 데이터/Training/원천데이터/TS1/05.minute/' + '**/*.json',
'AIHUB_요약문 및 레포트 생성 데이터/Training/원천데이터/TS1/06.edit/' + '**/*.json',
'AIHUB_요약문 및 레포트 생성 데이터/Training/원천데이터/TS1/07.public/' + '**/*.json',
'AIHUB_요약문 및 레포트 생성 데이터/Training/원천데이터/TS1/08.speech/' + '**/*.json',
'AIHUB_요약문 및 레포트 생성 데이터/Training/원천데이터/TS1/09.literature/' + '**/*.json',
'AIHUB_요약문 및 레포트 생성 데이터/Training/원천데이터/TS1/10.narration/' + '**/*.json',
'AIHUB_요약문 및 레포트 생성 데이터/Validation/원천데이터/VS1/01.news_r/' + '**/*.json',
'AIHUB_요약문 및 레포트 생성 데이터/Validation/원천데이터/VS1/02.briefing/' + '**/*.json',
'AIHUB_요약문 및 레포트 생성 데이터/Validation/원천데이터/VS1/03.his_cul/' + '**/*.json',
'AIHUB_요약문 및 레포트 생성 데이터/Validation/원천데이터/VS1/04.paper/' + '**/*.json',
'AIHUB_요약문 및 레포트 생성 데이터/Validation/원천데이터/VS1/05.minute/' + '**/*.json',
'AIHUB_요약문 및 레포트 생성 데이터/Validation/원천데이터/VS1/06.edit/' + '**/*.json',
'AIHUB_요약문 및 레포트 생성 데이터/Validation/원천데이터/VS1/07.public/' + '**/*.json',             
'AIHUB_요약문 및 레포트 생성 데이터/Validation/원천데이터/VS1/08.speech/' + '**/*.json',
'AIHUB_요약문 및 레포트 생성 데이터/Validation/원천데이터/VS1/09.literature/' + '**/*.json',
'AIHUB_요약문 및 레포트 생성 데이터/Validation/원천데이터/VS1/10.narration/' + '**/*.json']

train_file_name_01 = glob(path_list[0], recursive = True)
train_file_name_02 = glob(path_list[1], recursive = True)
train_file_name_03 = glob(path_list[2], recursive = True)
train_file_name_04 = glob(path_list[3], recursive = True)
train_file_name_05 = glob(path_list[4], recursive = True)
train_file_name_06 = glob(path_list[5], recursive = True)
train_file_name_07 = glob(path_list[6], recursive = True)
train_file_name_08 = glob(path_list[7], recursive = True)
train_file_name_09 = glob(path_list[8], recursive = True)
train_file_name_10 = glob(path_list[9], recursive = True)
valid_file_name_01 = glob(path_list[10], recursive = True)
valid_file_name_02 = glob(path_list[11], recursive = True)
valid_file_name_03 = glob(path_list[12], recursive = True)
valid_file_name_04 = glob(path_list[13], recursive = True)
valid_file_name_05 = glob(path_list[14], recursive = True)
valid_file_name_06 = glob(path_list[15], recursive = True)
valid_file_name_07 = glob(path_list[16], recursive = True)
valid_file_name_08 = glob(path_list[17], recursive = True)
valid_file_name_09 = glob(path_list[18], recursive = True)
valid_file_name_10 = glob(path_list[19], recursive = True)  

In [686]:
train_file_name_01[799]

'AIHUB_요약문 및 레포트 생성 데이터/Training/원천데이터/TS1/01.news_r\\20per\\REPORT-news_r-02275-00002.json'

In [355]:
with open(train_file_name_01[799], 'r', encoding='utf-8') as one_json_file:
    one_json_sample = json.load(one_json_file)

source = one_json_sample['Meta(Refine)']['passage']

In [356]:
source

' 하지만 크런치베이스에는 CB인사이츠에 포함된 바이오기업 에이프로젠 대신 이커머스 기업 티몬이 올라있다.\n  중소벤처기업부 등 국내 정부기관에서는 CB인사이츠의 자료를 활용한다.\n  정유신 서강대 경영대 교수는 "조사기관들은 보통 투자액을 통해 기업가치를 평가하는데, 투자액의 인정 기준이 조사기관마다 다를 수 있다"며 "글로벌 기준이 되는 미국의 조사결과를 더 신뢰하는 경향이 있다"고 설명했다.\n   ③국가 지원받는 중국, 데카콘만 6곳  CB인사이츠에서는 유니콘 기업의 분야를 16개로 나눴다.\n  핀테크 스타트업이 60곳으로 가장 많다.\n  이커머스(54곳)-인터넷서비스(54곳)-인공지능(46곳) 순이다.\n  미국 유니콘은 인터넷 서비스(43곳)-핀테크(30곳), 중국은 이커머스(21곳)-인공지능(12곳) 순이다.\n  한국은 쿠팡, 위메프, 무신사 등 이커머스 기업이 3곳으로 가장 많다.\n   기업가치가 유니콘의 10배 이상인 기업을 데카콘이라 부른다.\n  기업가치 100억 달러(11조 7000억원) 이상인 유니콘, 즉 데카콘은 총 23곳이다.\n  미국이 11곳이고, 중국이 6곳이다.\n  기업가치가 가장 높은 기업 1, 2위는 모두 중국 기업이다.\n  정 교수는 "중국에서는 기술 기업을 전폭적으로 지원해주고 있다"고 말했다.\n  1위는 동영상 플랫폼 \'틱톡\'을 서비스하는 바이트댄스(750억 달러), 2위는 \'중국의 우버\' 디디추싱(560억 달러)이다.\n  아시아 데카콘은 중국(6곳)을 포함해 인도 2곳, 인도네시아·싱가포르 1곳 등 10곳이다.\n  한국에선 쿠팡의 기업가치가 90억 달러(약 10조 500억원)로 가장 높다.\n   ④한국 10곳 중 5곳 적자   유니콘 기업이 잇달아 등장하자 정부에서는 적극적으로 육성 정책을 내놓고 있다.\n  하지만 전문가들은 유니콘에 대한 맹목적인 신뢰를 경계해야 한다고 말한다.\n  이병태 교수는 "유니콘 타이틀을 갖게 되면 언론의 주목을 받게 되고 추가 투자를 받기 유리해진다"며 "

In [357]:
preprocessing_sentences = formal_preprocessing_text(source)
for preprocessing_sentence in preprocessing_sentences:
    print(preprocessing_sentence)

하지만 크런치베이스에는 CB인사이츠에 포함된 바이오기업 에이프로젠 대신 이커머스 기업 티몬이 올라있다.
중소벤처기업부 등 국내 정부기관에서는 CB인사이츠의 자료를 활용한다.
정유신 서강대 경영대 교수는  조사기관들은 보통 투자액을 통해 기업가치를 평가하는데  투자액의 인정 기준이 조사기관마다 다를 수 있다 며  글로벌 기준이 되는 미국의 조사결과를 더 신뢰하는 경향이 있다 고 설명했다.
미국 유니콘은 인터넷 서비스(43곳)-핀테크(30곳)  중국은 이커머스(21곳)-인공지능(12곳) 순이다.
한국은 쿠팡  위메프  무신사 등 이커머스 기업이 3곳으로 가장 많다.
기업가치가 유니콘의 10배 이상인 기업을 데카콘이라 부른다.
기업가치 100억 달러(11조 7000억원) 이상인 유니콘  즉 데카콘은 총 23곳이다.
기업가치가 가장 높은 기업 1  2위는 모두 중국 기업이다.
정 교수는  중국에서는 기술 기업을 전폭적으로 지원해주고 있다 고 말했다.
1위는 동영상 플랫폼  틱톡 을 서비스하는 바이트댄스(750억 달러)  2위는  중국의 우버  디디추싱(560억 달러)이다.
아시아 데카콘은 중국(6곳)을 포함해 인도 2곳  인도네시아 싱가포르 1곳 등 10곳이다.
한국에선 쿠팡의 기업가치가 90억 달러(약 10조 500억원)로 가장 높다.
하지만 전문가들은 유니콘에 대한 맹목적인 신뢰를 경계해야 한다고 말한다.
이병태 교수는  유니콘 타이틀을 갖게 되면 언론의 주목을 받게 되고 추가 투자를 받기 유리해진다 며  기업 존립이 위태로운 기업이 유니콘으로 대접받는 경우도 있는데  거품이 끼어 있는 것 이라고 설명했다.
실제로 한국의 유니콘 10곳 가운데 5곳이 적자(2018년 기준 영업손실)다.
또 전 세계적으로 유니콘의 최소 기준인 기업가치 1억 달러 기업이 전체의 33 (146개)나 된다.
익명을 요청한 벤처캐피탈 관계자는  정부는 규제개혁을 통해 새로운 실험이 일어나게 하고  기초과학에 투자해 인적자산 확보에 나서야 한다.


In [128]:
train_file_name_09[932]

'AIHUB_요약문 및 레포트 생성 데이터/Training/원천데이터/TS1/09.literature\\20per\\REPORT-literature-08132-08132.json'

In [358]:
print(preprocessing_sentences)

['하지만 크런치베이스에는 CB인사이츠에 포함된 바이오기업 에이프로젠 대신 이커머스 기업 티몬이 올라있다.', '중소벤처기업부 등 국내 정부기관에서는 CB인사이츠의 자료를 활용한다.', '정유신 서강대 경영대 교수는  조사기관들은 보통 투자액을 통해 기업가치를 평가하는데  투자액의 인정 기준이 조사기관마다 다를 수 있다 며  글로벌 기준이 되는 미국의 조사결과를 더 신뢰하는 경향이 있다 고 설명했다.', '미국 유니콘은 인터넷 서비스(43곳)-핀테크(30곳)  중국은 이커머스(21곳)-인공지능(12곳) 순이다.', '한국은 쿠팡  위메프  무신사 등 이커머스 기업이 3곳으로 가장 많다.', '기업가치가 유니콘의 10배 이상인 기업을 데카콘이라 부른다.', '기업가치 100억 달러(11조 7000억원) 이상인 유니콘  즉 데카콘은 총 23곳이다.', '기업가치가 가장 높은 기업 1  2위는 모두 중국 기업이다.', '정 교수는  중국에서는 기술 기업을 전폭적으로 지원해주고 있다 고 말했다.', '1위는 동영상 플랫폼  틱톡 을 서비스하는 바이트댄스(750억 달러)  2위는  중국의 우버  디디추싱(560억 달러)이다.', '아시아 데카콘은 중국(6곳)을 포함해 인도 2곳  인도네시아 싱가포르 1곳 등 10곳이다.', '한국에선 쿠팡의 기업가치가 90억 달러(약 10조 500억원)로 가장 높다.', '하지만 전문가들은 유니콘에 대한 맹목적인 신뢰를 경계해야 한다고 말한다.', '이병태 교수는  유니콘 타이틀을 갖게 되면 언론의 주목을 받게 되고 추가 투자를 받기 유리해진다 며  기업 존립이 위태로운 기업이 유니콘으로 대접받는 경우도 있는데  거품이 끼어 있는 것 이라고 설명했다.', '실제로 한국의 유니콘 10곳 가운데 5곳이 적자(2018년 기준 영업손실)다.', '또 전 세계적으로 유니콘의 최소 기준인 기업가치 1억 달러 기업이 전체의 33 (146개)나 된다.', '익명을 요청한 벤처캐피탈 관계자는  정부는 규제개혁을 통해 새로운 실험이 일어나게 하고  기초

In [412]:
with open(train_file_name_09[932], 'r', encoding='utf-8') as one_json_file:
    one_json_sample = json.load(one_json_file)

source = one_json_sample['Meta(Refine)']['passage']

In [413]:
source

' 그러나 자연스런 그들의 행위가 반면에 의외로 큰 희생을 요구했으니 그것은 단체에 끼치게 된 불리보다도 참으로 크리이긴과 삑톨 두 사람에게 던지게 된 불행이다. 삑톨이 조바심을 하고 안달을 하면서 두 사람의 종적을 찾으러 휘돌아치는 꼴에는 단의 책임자로서의 심정보다도 마리이에게 대한 실망과 초조가 드러나 보이는 듯하다. 며칠 전 호텔 병실에서 그의 아내 그라아샤가 마리이를 냉큼 내쫓아 달라고 고함을 쳤던 것이 그럴 필요조차 없게 제물에 해결이 되어 마리이 쪽에서 마치 그 말을 엿듣기나 한 듯이 스스로 해결 짓게 된 것이 신통하다면 신통할까.\n그라아샤에게는 숨은 만족을 주었을 반면에 삑톨에게는 얼마나 큰 상처를 주었을까는 추측하기에 넉넉하다. 주체스런 몸을 이끌고 휘돌아치는 삑톨의 양이 딱하기 짝없는 것이었다. 그러나 삑톨보다도 한층 속이 타는 것은 크리이긴이 아니었을까. 아키임과 같은 모습이기는 하나 신경질이요 빳빳스런 그의 기질이 애태우고 맞서던 사랑을 뺏기고 얼마나 속이 휘둘리웠을까. 말하는 법 없이 고함치는 법 없이 더욱 벙어리같이 침묵해 가는 그의 마음속이 얼마나 울가망하고 답답한 것이었을까. 가령 나는 그의 옆을 지나는 길에 무어라고 한마디쯤 말을 걸어보려는 것이나 첫째 그의 시선을 잡을 수가 없는 것이다. 눈앞을 보지 않고 그 어디인지 먼 데를 보고 있다. 그리고 그 노리고 있는 한 가지에 생각에 열중해 있음은 그 우악스런 눈매와 모가 져 보이는 턱의 각도로 짐작할 수 있다. 아마도 마리이일 듯한 그 한 가지 환영에 불같이 마음을 뺏기고 있는 것이다.\n'

In [414]:
preprocessing_sentences = formal_preprocessing_text(source)
for preprocessing_sentence in preprocessing_sentences:
    print(preprocessing_sentence)

그러나 자연스런 그들의 행위가 반면에 의외로 큰 희생을 요구했으니 그것은 단체에 끼치게 된 불리보다도 참으로 크리이긴과 삑톨 두 사람에게 던지게 된 불행이다.
삑톨이 조바심을 하고 안달을 하면서 두 사람의 종적을 찾으러 휘돌아치는 꼴에는 단의 책임자로서의 심정보다도 마리이에게 대한 실망과 초조가 드러나 보이는 듯하다.
며칠 전 호텔 병실에서 그의 아내 그라아샤가 마리이를 냉큼 내쫓아 달라고 고함을 쳤던 것이 그럴 필요조차 없게 제물에 해결이 되어 마리이 쪽에서 마치 그 말을 엿듣기나 한 듯이 스스로 해결 짓게 된 것이 신통하다면 신통할까.
그라아샤에게는 숨은 만족을 주었을 반면에 삑톨에게는 얼마나 큰 상처를 주었을까는 추측하기에 넉넉하다.
주체스런 몸을 이끌고 휘돌아치는 삑톨의 양이 딱하기 짝없는 것이었다.
그러나 삑톨보다도 한층 속이 타는 것은 크리이긴이 아니었을까.
아키임과 같은 모습이기는 하나 신경질이요 빳빳스런 그의 기질이 애태우고 맞서던 사랑을 뺏기고 얼마나 속이 휘둘리웠을까  말하는 법 없이 고함치는 법 없이 더욱 벙어리같이 침묵해 가는 그의 마음속이 얼마나 울가망하고 답답한 것이었을까.
가령 나는 그의 옆을 지나는 길에 무어라고 한마디쯤 말을 걸어보려는 것이나 첫째 그의 시선을 잡을 수가 없는 것이다.
눈앞을 보지 않고 그 어디인지 먼 데를 보고 있다.
그리고 그 노리고 있는 한 가지에 생각에 열중해 있음은 그 우악스런 눈매와 모가 져 보이는 턱의 각도로 짐작할 수 있다.
아마도 마리이일 듯한 그 한 가지 환영에 불같이 마음을 뺏기고 있는 것이다.


In [415]:
print(preprocessing_sentences)

['그러나 자연스런 그들의 행위가 반면에 의외로 큰 희생을 요구했으니 그것은 단체에 끼치게 된 불리보다도 참으로 크리이긴과 삑톨 두 사람에게 던지게 된 불행이다.', '삑톨이 조바심을 하고 안달을 하면서 두 사람의 종적을 찾으러 휘돌아치는 꼴에는 단의 책임자로서의 심정보다도 마리이에게 대한 실망과 초조가 드러나 보이는 듯하다.', '며칠 전 호텔 병실에서 그의 아내 그라아샤가 마리이를 냉큼 내쫓아 달라고 고함을 쳤던 것이 그럴 필요조차 없게 제물에 해결이 되어 마리이 쪽에서 마치 그 말을 엿듣기나 한 듯이 스스로 해결 짓게 된 것이 신통하다면 신통할까.', '그라아샤에게는 숨은 만족을 주었을 반면에 삑톨에게는 얼마나 큰 상처를 주었을까는 추측하기에 넉넉하다.', '주체스런 몸을 이끌고 휘돌아치는 삑톨의 양이 딱하기 짝없는 것이었다.', '그러나 삑톨보다도 한층 속이 타는 것은 크리이긴이 아니었을까.', '아키임과 같은 모습이기는 하나 신경질이요 빳빳스런 그의 기질이 애태우고 맞서던 사랑을 뺏기고 얼마나 속이 휘둘리웠을까  말하는 법 없이 고함치는 법 없이 더욱 벙어리같이 침묵해 가는 그의 마음속이 얼마나 울가망하고 답답한 것이었을까.', '가령 나는 그의 옆을 지나는 길에 무어라고 한마디쯤 말을 걸어보려는 것이나 첫째 그의 시선을 잡을 수가 없는 것이다.', '눈앞을 보지 않고 그 어디인지 먼 데를 보고 있다.', '그리고 그 노리고 있는 한 가지에 생각에 열중해 있음은 그 우악스런 눈매와 모가 져 보이는 턱의 각도로 짐작할 수 있다.', '아마도 마리이일 듯한 그 한 가지 환영에 불같이 마음을 뺏기고 있는 것이다.']


### AIHUB 문서요약 텍스트

In [77]:
path_list = ['AIHUB_문서요약 텍스트/Training/'+ '/**/*.json', 
             'AIHUB_문서요약 텍스트/Validation/'+ '/**/*.json']
train_file_name, valid_file_name = train_valid_json_file_name_list(path_list)

In [75]:
train_file_name[1]

'AIHUB_문서요약 텍스트/Training\\사설_train_original\\train_original.json'

In [89]:
with open(train_file_name[1], 'r', encoding='utf-8') as one_json_file:
    one_json_sample = json.load(one_json_file)

In [ ]:
one_json_sample['documents'][1]

In [102]:
one_json_sample['documents'][1]['text'][1][1]

{'index': 4,
 'sentence': '현 정부 들어 제대로 된 남북대화 한 번 못했던 경색 국면은 꼭 풀어내야겠지만 그렇다고 원칙을 쉽게 내던져서도 곤란하다.',
 'highlight_indices': '0,1;8,11;19,20;23,24;34,35'}

In [103]:
for i in range(len(one_json_sample['documents'])):
    a = one_json_sample['documents'][i]
    for j in range(len(a['text'])):
        b = a['text'][j]
        for k in range(len(b)):
            c = b[k]['sentence']
            if '더구나 지방자치법' in c:
                print(i, " ",  j, " ",  k)

312   4   0
315   4   0


In [ ]:
one_json_sample['documents'][311]

In [76]:
with open(train_file_name[1], 'r', encoding='utf-8') as one_json_file:
    one_json_sample = json.load(one_json_file)
    
source_df = pd.DataFrame(one_json_sample['documents'])
source_dict = dict(source_df['text'].explode())
source_json = pd.json_normalize(source_dict)  

source_list = []
for source_index in source_json:
    for source_nested_list in source_json[source_index]:
        for source_single_list in source_nested_list:
            source_sentence = ""
            for source_single in source_single_list:
                if type(source_single) == dict:
                    for key, value in source_single.items():
                        if key == "sentence":
                            source_sentence += " " + value
            if len(source_sentence) > 0:
                source_list.append(source_sentence)

In [116]:
for source in range(len(source_list)):
    if '백화점들은 수입이 감소하지만 중소 입점업체' in source_list[source]:
        print(source)

1694
1710


In [122]:
source_list[1693]

' 공정거래위원회가 롯데 현대 신세계 등 빅3 백화점이 총 1054개 중소 납품업체 판매수수료율을 3~7%포인트씩 인하하고 이달분부터 소급 적용하기로 했다고 밝혔다. 인하 대상은 3대 백화점에 납품하는 중소업체 총 2036개 중 51.8%에 해당한다. 지난 9월 6일 김동수 공정위원장이 유통업계 대표들을 불러 수수료 인하를 독려한 후 인하폭을 둘러싸고 공정위와 백화점 간에 팽팽한 신경전이 펼쳐졌으나 공정위가 가이드라인으로 제시한 범위에서 합의가 이뤄졌다.'

In [114]:
source_list[1687]

' 중기 적합업종 제도가 소기의 성과를 내기 위해서는 대기업과 중소기업 간에 어느 정도 양해가 필요하다. 그런데 지난 1차 지정에서는 별 탈이 없었으나 이번 2차 지정에선 벌써부터 레미콘업계가 집단 반발 시위를 계획하는 등 말썽 소지가 벌어지고 있다. 동반성장위는 폼목을 늘리기 위해 졸속이 없었는지 살펴야 한다. 중소기업도 보호막에 안주하지 말고 위생, 안전성, 기술 품질, AS 등에서 소비자 만족도를 높이기 위한 노력을 배가해야 한다.'

In [123]:
preprocessing_sentences = formal_preprocessing_text(source_list[1693])
for preprocessing_sentence in preprocessing_sentences:
    print(preprocessing_sentence)

공정거래위원회가 롯데 현대 신세계 등 빅3 백화점이 총 1054개 중소 납품업체 판매수수료율을 3 7 포인트씩 인하하고 이달분부터 소급 적용하기로 했다고 밝혔다.
인하 대상은 3대 백화점에 납품하는 중소업체 총 2036개 중 51 8 에 해당한다.
지난 9월 6일 김동수 공정위원장이 유통업계 대표들을 불러 수수료 인하를 독려한 후 인하폭을 둘러싸고 공정위와 백화점 간에 팽팽한 신경전이 펼쳐졌으나 공정위가 가이드라인으로 제시한 범위에서 합의가 이뤄졌다.


In [124]:
print(preprocessing_sentences)

['공정거래위원회가 롯데 현대 신세계 등 빅3 백화점이 총 1054개 중소 납품업체 판매수수료율을 3 7 포인트씩 인하하고 이달분부터 소급 적용하기로 했다고 밝혔다.', '인하 대상은 3대 백화점에 납품하는 중소업체 총 2036개 중 51 8 에 해당한다.', '지난 9월 6일 김동수 공정위원장이 유통업계 대표들을 불러 수수료 인하를 독려한 후 인하폭을 둘러싸고 공정위와 백화점 간에 팽팽한 신경전이 펼쳐졌으나 공정위가 가이드라인으로 제시한 범위에서 합의가 이뤄졌다.']


In [125]:
with open(train_file_name[2], 'r', encoding='utf-8') as one_json_file:
    one_json_sample = json.load(one_json_file)
    
source_df = pd.DataFrame(one_json_sample['documents'])
source_dict = dict(source_df['text'].explode())
source_json = pd.json_normalize(source_dict)  

source_list = []
for source_index in source_json:
    for source_nested_list in source_json[source_index]:
        for source_single_list in source_nested_list:
            source_sentence = ""
            for source_single in source_single_list:
                if type(source_single) == dict:
                    for key, value in source_single.items():
                        if key == "sentence":
                            source_sentence += " " + value
            if len(source_sentence) > 0:
                source_list.append(source_sentence)

In [328]:
source_list[1899]

' 체 씨는“언어부터 문화까지 모든 게 달라 한국 사회에 적응하는 것이 쉽지 않았는데 광양제철소 사내 바리스타로 근무하기 시작하면서부터 삶이 달라졌다”라며“광양지역에 안정적으로 적응할 수 있도록 따뜻한 도움의 손길을 내밀어 준 광양제철소 프렌즈봉사단에 고맙다”고 말했다.'

In [329]:
preprocessing_sentences = formal_preprocessing_text(source_list[1899])
for preprocessing_sentence in preprocessing_sentences:
    print(preprocessing_sentence)

체 씨는 언어부터 문화까지 모든 게 달라 한국 사회에 적응하는 것이 쉽지 않았는데 광양제철소 사내 바리스타로 근무하기 시작하면서부터 삶이 달라졌다 라며 광양지역에 안정적으로 적응할 수 있도록 따뜻한 도움의 손길을 내밀어 준 광양제철소 프렌즈봉사단에 고맙다 고 말했다.


In [330]:
print(preprocessing_sentences)

['체 씨는 언어부터 문화까지 모든 게 달라 한국 사회에 적응하는 것이 쉽지 않았는데 광양제철소 사내 바리스타로 근무하기 시작하면서부터 삶이 달라졌다 라며 광양지역에 안정적으로 적응할 수 있도록 따뜻한 도움의 손길을 내밀어 준 광양제철소 프렌즈봉사단에 고맙다 고 말했다.']


### AIHUB 특허분야 자동분류 데이터

In [71]:
path_list = ['AIHUB_특허 분야 자동분류 데이터/Training/'+ '/**/*.json', 
             'AIHUB_특허 분야 자동분류 데이터/Validation/'+ '/**/*.json']
train_file_name, valid_file_name = train_valid_json_file_name_list(path_list)

In [72]:
with open(train_file_name[5], 'r', encoding='utf-8') as one_json_file:
    one_json_sample = json.load(one_json_file)

source_list = list(pd.DataFrame(one_json_sample['dataset'])['abstract'].dropna())

In [73]:
train_file_name[5]

'AIHUB_특허 분야 자동분류 데이터/Training\\원천데이터\\TS1\\A_농업_임업및어업_01_03\\01_농업\\011_작물재배업\\01132_음료용및향신용작물재배업.json'

In [261]:
source_list[74]

'본 발명은 옥수수 53.0 ∼ 57.0 중량%, 현미 22.0 ∼ 26.0 중량%, 결명자 2.2 ∼ 3.0 중량%, 둥굴레 2.2 ∼ 3.0 중량%, 치커리 2.2 ∼ 3.0 중량%, 감초 2.0 ∼ 4.0 중량%, 녹차잎 1.1 ∼ 1.5 중량%, 당귀 1.6 ∼ 2.0 중량%, 구기자 1.6 ∼ 2.0 중량%, 오미자 1.6 ∼ 2.0 중량%, 영지버섯 1.6 ∼ 2.0 중량%, 황기 1.6 ∼ 2.0 중량%를 포함하는 곡물을 주재로 한 음료 조성물에 관한 것이다.본 발명은 옥수수, 현미 및 그 외 둥굴레 등의 영양과 맛이 잘 조화되어 맛과 향기면에 있어서 그 기호도가 매우 높고, 옥수수를 주재로 하고 여기에 현미, 둥굴레, 치커리, 오미자, 구기자, 녹차잎, 감초, 당귀, 영지버섯, 황기, 결명자 등을 넣어 끓인 물을 매일 지속적으로 마실 수 있도록 하여 음료 시장 개선과 건강 증진에 기여하며, 티백 포장함으로써 사용이 간편할 뿐 아니라, 원료 상태부터 포장까지의 전공정이 소비자 앞에서 이루어짐으로써 음료 조성물에 대한 소비자의 신뢰성을 극대화하는 효과가 있다.'

In [262]:
preprocessing_sentences = formal_preprocessing_text(source_list[74])
for preprocessing_sentence in preprocessing_sentences:
    print(preprocessing_sentence)

본 발명은 옥수수 53 0   57 0 중량   현미 22 0   26 0 중량   결명자 2 2   3 0 중량   둥굴레 2 2   3 0 중량   치커리 2 2   3 0 중량   감초 2 0   4 0 중량   녹차잎 1 1   1 5 중량   당귀 1 6   2 0 중량   구기자 1 6   2 0 중량   오미자 1 6   2 0 중량   영지버섯 1 6   2 0 중량   황기 1 6   2 0 중량 를 포함하는 곡물을 주재로 한 음료 조성물에 관한 것이다.
본 발명은 옥수수  현미 및 그 외 둥굴레 등의 영양과 맛이 잘 조화되어 맛과 향기면에 있어서 그 기호도가 매우 높고  옥수수를 주재로 하고 여기에 현미  둥굴레  치커리  오미자  구기자  녹차잎  감초  당귀  영지버섯  황기  결명자 등을 넣어 끓인 물을 매일 지속적으로 마실 수 있도록 하여 음료 시장 개선과 건강 증진에 기여하며  티백 포장함으로써 사용이 간편할 뿐 아니라  원료 상태부터 포장까지의 전공정이 소비자 앞에서 이루어짐으로써 음료 조성물에 대한 소비자의 신뢰성을 극대화하는 효과가 있다.


In [263]:
print(preprocessing_sentences)

['본 발명은 옥수수 53 0   57 0 중량   현미 22 0   26 0 중량   결명자 2 2   3 0 중량   둥굴레 2 2   3 0 중량   치커리 2 2   3 0 중량   감초 2 0   4 0 중량   녹차잎 1 1   1 5 중량   당귀 1 6   2 0 중량   구기자 1 6   2 0 중량   오미자 1 6   2 0 중량   영지버섯 1 6   2 0 중량   황기 1 6   2 0 중량 를 포함하는 곡물을 주재로 한 음료 조성물에 관한 것이다.', '본 발명은 옥수수  현미 및 그 외 둥굴레 등의 영양과 맛이 잘 조화되어 맛과 향기면에 있어서 그 기호도가 매우 높고  옥수수를 주재로 하고 여기에 현미  둥굴레  치커리  오미자  구기자  녹차잎  감초  당귀  영지버섯  황기  결명자 등을 넣어 끓인 물을 매일 지속적으로 마실 수 있도록 하여 음료 시장 개선과 건강 증진에 기여하며  티백 포장함으로써 사용이 간편할 뿐 아니라  원료 상태부터 포장까지의 전공정이 소비자 앞에서 이루어짐으로써 음료 조성물에 대한 소비자의 신뢰성을 극대화하는 효과가 있다.']


### AIHUB 뉴스기사 기계독해 데이터

In [65]:
path_list = ['AIHUB_뉴스 기사 기계독해 데이터/Training/'+ '/**/*.json', 
             'AIHUB_뉴스 기사 기계독해 데이터/Validation/'+ '/**/*.json']
train_file_name, valid_file_name = train_valid_json_file_name_list(path_list)

In [66]:
with open(train_file_name[0], 'r', encoding='utf-8') as one_json_file:
    one_json_sample = json.load(one_json_file)

source_df = pd.DataFrame(one_json_sample['data'])
source_dict = dict(source_df['paragraphs'].explode())
source_json = pd.json_normalize(source_dict)
source_list = list(source_json.filter(regex='context').values[0])

In [68]:
for i in range(len(one_json_sample['data'])):
    if '군산시는 노인 복지향상 ' in one_json_sample['data'][i]['paragraphs'][0]['context']:
        print(i)

8


In [ ]:
one_json_sample

In [69]:
one_json_sample['data'][8]

{'doc_id': '01601101.20210325201931006',
 'doc_title': '군산시, 도내 최초 경로당 친환경쌀 공급',
 'doc_source': '전북일보',
 'doc_published': 20210325,
 'doc_class': {'class': '한국언론진흥재단 빅카인즈 뉴스기사', 'code': '지역'},
 'created': '20211217142431',
 'paragraphs': [{'context': '군산시는 노인 복지향상 및 건강증진 도모를 위해 도내 최초로 경로당 친환경쌀 공급 지원사업을 추진한다. \n\n최근 노인들의 경로당 참여율이 높아지면서 쌀 소비량이 늘어나고 노인들의 건강한 지역 먹거리에 대한 욕구도 증대되고 있는 상황이다. \n\n이에 따라 시는 경로장애인과와 협력해 경로당에 공급하는 기존 정부 양곡을 지역에서 생산된 우수한 품질의 친환경쌀로 대체 지원하기로 했다. \n\n시는 향후 지역 경로당 516곳에 총 4128포(20kg)의 친환경쌀을 경로당 희망하는 월(月)에 공급할 계획이다. \n\n시는 이번 지원 사업이 경로당 이용 노인들의 건강한 식생활을 돕고, 군산 친환경쌀 판매 확대를 통해 쌀 생산 농가의 소득증대와 지역경제 활성화에 기여 할 것으로 보고 있다. \n\n신동우 군산시 먹거리정책과장은 “경로당뿐만 아니라 공공급식 영역을 더욱 확대해 안전하고 우수한 우리지역 농산물을 공급하는데 노력하겠다”고 말했다.',
   'context_id': '464138'}]}

In [67]:
train_file_name[0]

'AIHUB_뉴스 기사 기계독해 데이터/Training\\원천데이터\\TS_span_extraction.json'

In [246]:
source_list[16]

'군산시는 노인 복지향상 및 건강증진 도모를 위해 도내 최초로 경로당 친환경쌀 공급 지원사업을 추진한다. \n\n최근 노인들의 경로당 참여율이 높아지면서 쌀 소비량이 늘어나고 노인들의 건강한 지역 먹거리에 대한 욕구도 증대되고 있는 상황이다. \n\n이에 따라 시는 경로장애인과와 협력해 경로당에 공급하는 기존 정부 양곡을 지역에서 생산된 우수한 품질의 친환경쌀로 대체 지원하기로 했다. \n\n시는 향후 지역 경로당 516곳에 총 4128포(20kg)의 친환경쌀을 경로당 희망하는 월(月)에 공급할 계획이다. \n\n시는 이번 지원 사업이 경로당 이용 노인들의 건강한 식생활을 돕고, 군산 친환경쌀 판매 확대를 통해 쌀 생산 농가의 소득증대와 지역경제 활성화에 기여 할 것으로 보고 있다. \n\n신동우 군산시 먹거리정책과장은 “경로당뿐만 아니라 공공급식 영역을 더욱 확대해 안전하고 우수한 우리지역 농산물을 공급하는데 노력하겠다”고 말했다.'

In [245]:
preprocessing_sentences = formal_preprocessing_text(source_list[16])
for preprocessing_sentence in preprocessing_sentences:
    print(preprocessing_sentence)

군산시는 노인 복지향상 및 건강증진 도모를 위해 도내 최초로 경로당 친환경쌀 공급 지원사업을 추진한다.
최근 노인들의 경로당 참여율이 높아지면서 쌀 소비량이 늘어나고 노인들의 건강한 지역 먹거리에 대한 욕구도 증대되고 있는 상황이다.
이에 따라 시는 경로장애인과와 협력해 경로당에 공급하는 기존 정부 양곡을 지역에서 생산된 우수한 품질의 친환경쌀로 대체 지원하기로 했다.
시는 향후 지역 경로당 516곳에 총 4128포(20kg)의 친환경쌀을 경로당 희망하는 월(月)에 공급할 계획이다.
시는 이번 지원 사업이 경로당 이용 노인들의 건강한 식생활을 돕고  군산 친환경쌀 판매 확대를 통해 쌀 생산 농가의 소득증대와 지역경제 활성화에 기여 할 것으로 보고 있다.
신동우 군산시 먹거리정책과장은  경로당뿐만 아니라 공공급식 영역을 더욱 확대해 안전하고 우수한 우리지역 농산물을 공급하는데 노력하겠다 고 말했다.


In [247]:
print(preprocessing_sentences)

['군산시는 노인 복지향상 및 건강증진 도모를 위해 도내 최초로 경로당 친환경쌀 공급 지원사업을 추진한다.', '최근 노인들의 경로당 참여율이 높아지면서 쌀 소비량이 늘어나고 노인들의 건강한 지역 먹거리에 대한 욕구도 증대되고 있는 상황이다.', '이에 따라 시는 경로장애인과와 협력해 경로당에 공급하는 기존 정부 양곡을 지역에서 생산된 우수한 품질의 친환경쌀로 대체 지원하기로 했다.', '시는 향후 지역 경로당 516곳에 총 4128포(20kg)의 친환경쌀을 경로당 희망하는 월(月)에 공급할 계획이다.', '시는 이번 지원 사업이 경로당 이용 노인들의 건강한 식생활을 돕고  군산 친환경쌀 판매 확대를 통해 쌀 생산 농가의 소득증대와 지역경제 활성화에 기여 할 것으로 보고 있다.', '신동우 군산시 먹거리정책과장은  경로당뿐만 아니라 공공급식 영역을 더욱 확대해 안전하고 우수한 우리지역 농산물을 공급하는데 노력하겠다 고 말했다.']


### AIHUB 행정 문서 대상 기계독해 데이터

In [237]:
def preprocessing_text(source):
    
    preprocessing_sentence_list = []
    source_split = re.split('br|tr|td|table|tbody|rowspan', source)
    # 문자열 'br', 'tr', 'td', 'table', 'tbody'를 기준으로 문장 분리
    
    for source in source_split:
        sentence = re.sub('(br|tr|td|table|tbody|rowspan|<|>|/)', '', source)
        # 문자열 'br', 'tr', 'td', 'table', 'tbody' 특수기호 '<', '>', '/'를 제외
            
        sentence = sentence.strip()
        # strip으로 앞뒤 공백 제거

        if len(sentence) > 1 and \
        re.search("^[A-Za-z0-9ㄱ-ㅎ가-힣一-鿕㐀-䶵豈-龎]", sentence[0]) is not None and \
        bool(re.match(r'[.]|[!]|[?]', sentence[-1])) == True and  \
        len(sentence.split()) > 5:
            # The String starts with a letter
            # The String ends with [. ! ?]

            sentence = re.sub(r"[^A-Za-z0-9ㄱ-ㅎ가-힣一-鿕㐀-䶵豈-龎()+-]", " ", sentence)
            # 특수문자 제거(영어 대소문자, 한글, 한자, 숫자, -, +, 소괄호 제외)
            #               
            sentence = re.sub(r"\[.*?\]|\{.*?\}", "",  sentence)
            # 기타 괄호 제거할 시 괄호 내부에 모든 텍스트 제거  

            sentence = sentence.strip()
            # strip으로 앞뒤 공백 제거
            
            total_length = len(sentence.replace(" " , ""))
            hangeul_length = len(re.sub(r"[^ㄱ-ㅣ가-힣\s]", "", sentence.replace(" " , "")))
            hangeul_ratio = hangeul_length / total_length
            if hangeul_ratio >= 0.5:
            # 한글이 아닌 문자열이 50% 이상이 넘은 문장 제외
                if sentence[-1] == ".":
                    preprocessing_sentence_list.append(sentence)
                else:
                    preprocessing_sentence_list.append(sentence + ".")
                    
    return preprocessing_sentence_list

In [50]:
path_list = ['AIHUB_행정 문서 대상 기계독해 데이터/Training/'+ '/**/*.json', 
             'AIHUB_행정 문서 대상 기계독해 데이터/Validation/'+ '/**/*.json']
train_file_name, valid_file_name = train_valid_json_file_name_list(path_list)

In [60]:
with open(train_file_name[0], 'r', encoding='utf-8') as one_json_file:
    one_json_sample = json.load(one_json_file) 

source_df = pd.DataFrame(one_json_sample['data'])
source_dict = dict(source_df['paragraphs'].explode())
source_json = pd.json_normalize(source_dict)
source_list = list(source_json.filter(regex='context').values[0])

In [61]:
for i in range(len(one_json_sample['data'])):
    if '신년 업무' in one_json_sample['data'][i]['paragraphs'][0]['context']:
        print(i)

11
577


In [ ]:
one_json_sample

In [63]:
one_json_sample['data'][11]

{'doc_id': 'D0000042797225',
 'doc_title': '2021년 6월 신규공사장 안전기술지도 시행계획 변경(교육현장 추가)',
 'doc_source': '서울특별시청',
 'doc_published': 20210618,
 'doc_class': {'class': '중앙행정기관 분류체계', 'code': '산업고용'},
 'created': '20220422145553',
 'paragraphs': [{'context': '제목 2021년 6월 신규공사장 안전기술지도 시행계획 변경\n1. 2021년 본부장 신년 업무보고(2021.1.27.)와 관련하여 「신규공사장 안전기술지도」2021년 6월 시행계획을 다음과 같이 보고드립니다.\n 가. 지도일정 : 2021년 6월 10일(목), 29일(화) 오전\n 나. 지도강사 : 시설국장, 안전관리과장, 안전관리과 정진혁, 이태행\n 다. 교육시간 : 오전 09:30 ~ 12:30(3.0H)\n 라. 교육장소 : 도시기반시설본부 10층 회의실\n 마. 교육대상 : 현장소장, 감리단장, 안전관리자 등\n 바. 교육내용\n 1) 건설공사장 안전관리 방향\n 2) 건설기술진흥법, 산업안전보건법 상의 안전관리 내용과 구비 및 관리 서류(1.5H)\n 3) 도시기반시설본부 건설안전 관련 각종 방침 및 지침',
   'context_id': '398205'}]}

In [221]:
source_list[22]

'제목 2021년 6월 신규공사장 안전기술지도 시행계획 변경\n1. 2021년 본부장 신년 업무보고(2021.1.27.)와 관련하여 「신규공사장 안전기술지도」2021년 6월 시행계획을 다음과 같이 보고드립니다.\n 가. 지도일정 : 2021년 6월 10일(목), 29일(화) 오전\n 나. 지도강사 : 시설국장, 안전관리과장, 안전관리과 정진혁, 이태행\n 다. 교육시간 : 오전 09:30 ~ 12:30(3.0H)\n 라. 교육장소 : 도시기반시설본부 10층 회의실\n 마. 교육대상 : 현장소장, 감리단장, 안전관리자 등\n 바. 교육내용\n 1) 건설공사장 안전관리 방향\n 2) 건설기술진흥법, 산업안전보건법 상의 안전관리 내용과 구비 및 관리 서류(1.5H)\n 3) 도시기반시설본부 건설안전 관련 각종 방침 및 지침'

In [220]:
preprocessing_sentences = formal_preprocessing_text(source_list[22])
for preprocessing_sentence in preprocessing_sentences:
    print(preprocessing_sentence)

1  2021년 본부장 신년 업무보고(2021 1 27 )와 관련하여  신규공사장 안전기술지도 2021년 6월 시행계획을 다음과 같이 보고드립니다.


In [222]:
print(preprocessing_sentences)

['1  2021년 본부장 신년 업무보고(2021 1 27 )와 관련하여  신규공사장 안전기술지도 2021년 6월 시행계획을 다음과 같이 보고드립니다.']


In [223]:
train_file_name[3]

'AIHUB_행정 문서 대상 기계독해 데이터/Training\\원천데이터\\TS_tableqa.json'

In [52]:
with open(train_file_name[3], 'r', encoding='utf-8') as one_json_file:
    one_json_sample = json.load(one_json_file) 

source_df = pd.DataFrame(one_json_sample['data'])
source_dict = dict(source_df['paragraphs'].explode())
source_json = pd.json_normalize(source_dict)
source_list = list(source_json.filter(regex='context').values[0])

In [53]:
for i in range(len(one_json_sample['data'])):
    if '여름철 고압가스 충전' in one_json_sample['data'][i]['paragraphs'][0]['context']:
        print(i)

21


In [ ]:
one_json_sample

In [ ]:
one_json_sample['data'][21]

In [239]:
source_list[42]

'제  목 2021년 여름철 고압가스 충전시설 안전점검(자체) 계획 알림<br>1. 은평구청 환경과-18398(2021.6.22.)호『2021년 여름철 고압가스 충전시설 안전점검 결과 제출 요청』과 관련입니다.<br>2. 2021년 여름철 가스시설 안전점검 계획에 따라 고압가스 관련시설에 대한 자체안전점검 실시 후 결과를 2021.7.28.(수)까지 제출하여 주시기 바랍니다. <table><tbody><tr><td>연번</td><td>시설명</td><td>가스종류</td><td>소재지</td><td>비고</td></tr><tr><td>1</td><td>은평소방서(현장대응단)</td><td>공기압축(호흡기용 공기충전기)</td><td>통일로962</td><td></td></tr><tr><td>2</td><td>은평소방서(수색119안전센터)</td><td>공기압축(호흡기용 공기충전기)</td><td>수색로320</td><td></td></tr></tbody></table>'

In [238]:
preprocessing_sentences = preprocessing_text(source_list[42])
for preprocessing_sentence in preprocessing_sentences:
    print(preprocessing_sentence)

1  은평구청 환경과-18398(2021 6 22 )호 2021년 여름철 고압가스 충전시설 안전점검 결과 제출 요청 과 관련입니다.
2  2021년 여름철 가스시설 안전점검 계획에 따라 고압가스 관련시설에 대한 자체안전점검 실시 후 결과를 2021 7 28 (수)까지 제출하여 주시기 바랍니다.


In [240]:
print(preprocessing_sentences)

['1  은평구청 환경과-18398(2021 6 22 )호 2021년 여름철 고압가스 충전시설 안전점검 결과 제출 요청 과 관련입니다.', '2  2021년 여름철 가스시설 안전점검 계획에 따라 고압가스 관련시설에 대한 자체안전점검 실시 후 결과를 2021 7 28 (수)까지 제출하여 주시기 바랍니다.']


### AIHUB 기계독해

In [41]:
path_list = ['AIHUB_기계독해'+ '/**/*.json']
file_name = json_file_name_list(path_list)

In [44]:
with open(file_name[0], 'r', encoding='utf-8') as one_json_file:
    one_json_sample = json.load(one_json_file)

source_df = pd.DataFrame(one_json_sample['data'])
source_dict = dict(source_df['paragraphs'].explode())
source_json = pd.json_normalize(source_dict)
source_list = list(source_json.filter(regex='context').values[0])

In [46]:
for i in range(len(one_json_sample['data'])):
    if '시청률 30%를 목전에 두고 있다.' in one_json_sample['data'][i]['paragraphs'][0]['context']:
        print(i)

55
29187


In [49]:
one_json_sample['data'][55]

{'source': 8,
 'paragraphs': [{'qas': [{'classtype': 'work_who',
     'id': 'm6_426078-1',
     'answers': [{'text': '27.7%', 'answer_start': 112}],
     'question': "지난 1일 방송된 '같이 살래요'의 전국 기준 시청률이 얼마야?",
     'clue': [{'clue_start': 125,
       'clue_text': '이는 지난 방송분 21.8%에 비해 5.9% 포인트 상승한 수치다.'}]},
    {'classtype': 'work_who',
     'id': 'm6_426078-2',
     'answers': [{'text': '27.7%', 'answer_start': 112}],
     'question': "지난 1일 방송된 '같이 살래요'의 전국 기준 시청률이 얼마인거야?",
     'clue': [{'clue_start': 125,
       'clue_text': '이는 지난 방송분 21.8%에 비해 5.9% 포인트 상승한 수치다.'}]}],
   'context': '▲ 사진= KBS2 같이 살래요 영상캡처 ‘같이 살래요’가 시청률 30%를 목전에 두고 있다. 2일 시청률조사회사 닐슨코리아에 따르면 지난 1일 방송된 KBS2 주말드라마 ‘같이 살래요’는 전국 기준 27.7%를 기록했다. 이는 지난 방송분 21.8%에 비해 5.9% 포인트 상승한 수치다. 이는 같이 살래요의 자체최고 시청률이기도 하다. 이날 방송에서는 박효섭(유동근 분)의 빚으로 가족들이 위기를 맞았다. 또한 박효섭에게 복수하기 위해 상가거리 재개발을 진행했던 이미연(장미희 분)은 그간의 추억에 빠지며, 사업을 지속할지 궁금증을 자아냈다. 한편 동시간대 방송된 MBC 주말드라마 ‘부잣집 아들’은 4.3%, 11.2%, 9.9%, 10.5%를 나타내며 같이 살래요와 큰 격차를 보였다. 압도적인 상승세를 보인 같이 살래요가 시청

In [47]:
one_json_sample['data'][55]

{'source': 8,
 'paragraphs': [{'qas': [{'classtype': 'work_who',
     'id': 'm6_426078-1',
     'answers': [{'text': '27.7%', 'answer_start': 112}],
     'question': "지난 1일 방송된 '같이 살래요'의 전국 기준 시청률이 얼마야?",
     'clue': [{'clue_start': 125,
       'clue_text': '이는 지난 방송분 21.8%에 비해 5.9% 포인트 상승한 수치다.'}]},
    {'classtype': 'work_who',
     'id': 'm6_426078-2',
     'answers': [{'text': '27.7%', 'answer_start': 112}],
     'question': "지난 1일 방송된 '같이 살래요'의 전국 기준 시청률이 얼마인거야?",
     'clue': [{'clue_start': 125,
       'clue_text': '이는 지난 방송분 21.8%에 비해 5.9% 포인트 상승한 수치다.'}]}],
   'context': '▲ 사진= KBS2 같이 살래요 영상캡처 ‘같이 살래요’가 시청률 30%를 목전에 두고 있다. 2일 시청률조사회사 닐슨코리아에 따르면 지난 1일 방송된 KBS2 주말드라마 ‘같이 살래요’는 전국 기준 27.7%를 기록했다. 이는 지난 방송분 21.8%에 비해 5.9% 포인트 상승한 수치다. 이는 같이 살래요의 자체최고 시청률이기도 하다. 이날 방송에서는 박효섭(유동근 분)의 빚으로 가족들이 위기를 맞았다. 또한 박효섭에게 복수하기 위해 상가거리 재개발을 진행했던 이미연(장미희 분)은 그간의 추억에 빠지며, 사업을 지속할지 궁금증을 자아냈다. 한편 동시간대 방송된 MBC 주말드라마 ‘부잣집 아들’은 4.3%, 11.2%, 9.9%, 10.5%를 나타내며 같이 살래요와 큰 격차를 보였다. 압도적인 상승세를 보인 같이 살래요가 시청

In [43]:
file_name[0]

'AIHUB_기계독해\\ko_nia_clue0529_squad_all.json'

In [193]:
source_list[55]

'▲ 사진= KBS2 같이 살래요 영상캡처 ‘같이 살래요’가 시청률 30%를 목전에 두고 있다. 2일 시청률조사회사 닐슨코리아에 따르면 지난 1일 방송된 KBS2 주말드라마 ‘같이 살래요’는 전국 기준 27.7%를 기록했다. 이는 지난 방송분 21.8%에 비해 5.9% 포인트 상승한 수치다. 이는 같이 살래요의 자체최고 시청률이기도 하다. 이날 방송에서는 박효섭(유동근 분)의 빚으로 가족들이 위기를 맞았다. 또한 박효섭에게 복수하기 위해 상가거리 재개발을 진행했던 이미연(장미희 분)은 그간의 추억에 빠지며, 사업을 지속할지 궁금증을 자아냈다. 한편 동시간대 방송된 MBC 주말드라마 ‘부잣집 아들’은 4.3%, 11.2%, 9.9%, 10.5%를 나타내며 같이 살래요와 큰 격차를 보였다. 압도적인 상승세를 보인 같이 살래요가 시청률 30%까지 돌파할 수 있을지 귀추가 주목된다. /ekqls_star@fnnews.com fn스타 우다빈 기자'

In [194]:
formal_preprocessing_text(source_list[55])

['2일 시청률조사회사 닐슨코리아에 따르면 지난 1일 방송된 KBS2 주말드라마  같이 살래요 는 전국 기준 27 7 를 기록했다.',
 '이는 지난 방송분 21 8 에 비해 5 9  포인트 상승한 수치다.',
 '이는 같이 살래요의 자체최고 시청률이기도 하다.',
 '이날 방송에서는 박효섭(유동근 분)의 빚으로 가족들이 위기를 맞았다.',
 '또한 박효섭에게 복수하기 위해 상가거리 재개발을 진행했던 이미연(장미희 분)은 그간의 추억에 빠지며  사업을 지속할지 궁금증을 자아냈다.',
 '한편 동시간대 방송된 MBC 주말드라마  부잣집 아들 은 4 3   11 2   9 9   10 5 를 나타내며 같이 살래요와 큰 격차를 보였다.',
 '압도적인 상승세를 보인 같이 살래요가 시청률 30 까지 돌파할 수 있을지 귀추가 주목된다.']

### AIHUB 도서자료 요약

In [38]:
path_list = ['AIHUB_도서자료 요약/Training/'+ '/**/*.json', 
             'AIHUB_도서자료 요약/Validation/'+ '/**/*.json']
train_file_name, valid_file_name = train_valid_json_file_name_list(path_list)

In [40]:
train_file_name[89900]

'AIHUB_도서자료 요약/Training\\[원천]도서요약_train\\사회과학\\PCY_201612260512486961_34.json'

In [169]:
with open(train_file_name[89900], 'r', encoding='utf-8') as one_json_file:
    one_json_sample = json.load(one_json_file)

sentence_list = []
passage = one_json_sample["passage"]  
summary = one_json_sample["summary"]

passage_sentences = formal_preprocessing_text(passage)
for sentence in passage_sentences:
    sentence_list.append(sentence)   

summary_sentences = formal_preprocessing_text(summary)
for sentence in summary_sentences:
    sentence_list.append(sentence) 

In [171]:
passage

'임상사회복지 분야에서 1979년에서 1999년 사이에 생산된 논문들을 연구한 양옥경(2000)에 따르면, 프로그램의 효과를 평가하는 연구는 그 출현율이 1.28%에 그쳤고, 이 중 실험/통제집단 설계를 사용한 경우는 22%에 지나지 않은 것으로 나타났다. 다시 말해서 이 분야에서 실험/통제집단 설계를 갖춘 평가연구의 출현율은 0.3%미만이라는 것을 알 수 있다(.0128*.22=0.0028). 이처럼 다양한 프로그램이 시행되는 사회복지분야에서도 증거의 양적, 질적 수준은 매우 낮은 수준이다. 따라서 사회복지 분야에서 증거기반 정책의 국내 적용가능성에 대해 평가한 공계순과 서인해(2005)는 외국의 평가연구결과를 사회환경적 차이를 감안한 후에 사용할 것을 제안한다. 양적 방법론이 많이 발달된 범죄학 분야는 사회복지분야에 비해 그 사정이 다소 낮겠으나, 그 열악한 환경에서 큰 차이는 없는 것으로 판단된다.'

In [170]:
sentence_list

['임상사회복지 분야에서 1979년에서 1999년 사이에 생산된 논문들을 연구한 양옥경(2000)에 따르면  프로그램의 효과를 평가하는 연구는 그 출현율이 1 28 에 그쳤고  이 중 실험 통제집단 설계를 사용한 경우는 22 에 지나지 않은 것으로 나타났다.',
 '다시 말해서 이 분야에서 실험 통제집단 설계를 갖춘 평가연구의 출현율은 0 3 미만이라는 것을 알 수 있다( 0128  22 0 0028)  이처럼 다양한 프로그램이 시행되는 사회복지분야에서도 증거의 양적  질적 수준은 매우 낮은 수준이다.',
 '따라서 사회복지 분야에서 증거기반 정책의 국내 적용가능성에 대해 평가한 공계순과 서인해(2005)는 외국의 평가연구결과를 사회환경적 차이를 감안한 후에 사용할 것을 제안한다.',
 '양적 방법론이 많이 발달된 범죄학 분야는 사회복지분야에 비해 그 사정이 다소 낮겠으나  그 열악한 환경에서 큰 차이는 없는 것으로 판단된다.',
 '한국 내의 사회복지 분야에서 증거기반 정책의 사용은 해외의 연구 결과를 사회환경적 차이를 고려한 후에 사용해야 한다.']

In [ ]:
for strange_source in strange_source_list:
    print(strange_source, end="\n\n")

### AIHUB 대규모 구매도서 기반 한국어 말뭉치 데이터

In [36]:
path_list = ['AIHUB_대규모 구매도서 기반 한국어 말뭉치 데이터/sample/라벨링데이터/000/'+ '/**/*.json']
file_name = json_file_name_list(path_list)
for i in file_name:
    if 'INFO' in i:
        file_name.remove(i)

In [59]:
with open(file_name[0], 'r', encoding='utf-8') as one_json_file:
    one_json_sample = json.load(one_json_file)
    
source_df = pd.DataFrame(one_json_sample['paragraphs'])
source_dict = dict(source_df['sentences'].explode())
source_json = pd.json_normalize(source_dict)  

source_list = []
for source_index in source_json:
    for source_nested_list in source_json[source_index]:

        try:
            for source_single_list in source_nested_list:
                try:
                    for key, value in source_single_list.items():
                        if key == 'text':
                            source_list.append(value)
                except:
                    pass
        except:
            pass  

In [37]:
file_name[0]

'AIHUB_대규모 구매도서 기반 한국어 말뭉치 데이터/sample/라벨링데이터/000\\BOOK_CORPUS_000_TEXT_0001.json'

In [90]:
source_list[300]

'새로 조, 경 단위의 부자가 된 사람들은 유기농 “공정거래” 인간 노동으로 만든 상품과 서비스에 엄청난 프리미엄을 지불할 능력이 될 것이다.'

In [ ]:
'새로 조, 경 단위의 부자가 된 사람들은 유기농 “공정거래” 인간 노동으로 만든 상품과 서비스에 엄청난 프리미엄을 지불할 능력이 될 것이다.'

In [92]:
strange_source = '새로 조, 경 단위의 부자가 된 사람들은 유기농 “공정거래” 인간 노동으로 만든 상품과 서비스에 엄청난 프리미엄을 지불할 능력이 될 것이다.'
pprint(strange_source)

'새로 조, 경 단위의 부자가 된 사람들은 유기농 “공정거래” 인간 노동으로 만든 상품과 서비스에 엄청난 프리미엄을 지불할 능력이 될 것이다.'


In [93]:
preprocessing_sentences = formal_preprocessing_text(source_list[300])
for preprocessing_sentence in preprocessing_sentences:
    print(preprocessing_sentence)

새로 조  경 단위의 부자가 된 사람들은 유기농  공정거래  인간 노동으로 만든 상품과 서비스에 엄청난 프리미엄을 지불할 능력이 될 것이다.


In [94]:
print(preprocessing_sentences)

['새로 조  경 단위의 부자가 된 사람들은 유기농  공정거래  인간 노동으로 만든 상품과 서비스에 엄청난 프리미엄을 지불할 능력이 될 것이다.']


### AIHUB 도서자료 기계독해

In [28]:
path_list = ['AIHUB_도서자료 기계독해/'+ '/**/*.json']
file_name = json_file_name_list(path_list)

In [29]:
file_name

['AIHUB_도서자료 기계독해\\Training\\도서_220419_add\\도서_220419_add.json',
 'AIHUB_도서자료 기계독해\\Validation\\도서.json']

In [30]:
file_name[0]

'AIHUB_도서자료 기계독해\\Training\\도서_220419_add\\도서_220419_add.json'

In [32]:
for i in range(len(one_json_sample['data'])):
    if '흔하지는 않지만' in one_json_sample['data'][i]['paragraphs'][0]['context']:
        print(i)

2594


In [33]:
one_json_sample['data'][2594]['paragraphs'][0]['context']

'5) 흔하지는 않지만, 어떤 약물이상반응은 시간에 따라 빈도나 중증도가 증가할 수 있으며, 어떤 심각한 약물이상반응은 6개월 이상의 투약 이후에만 발생할 수도 있다. 그러므로 일부 환자들은 12개월 동안 약물이 투여되어야 한다. 투여기간과 약물이상반응의 상관성에 대해 충분한 정보가 없을 경우, 1년간 추적할 특정 환자수를 선정하는 것은 대부분, 약물이상반응의 빈도 수준을 발견할 확률과 실제적인 고려사항을 근거로 결정된다. 최소 1년간 약물이 투여된 100명의 환자에서 얻은 자료는 안전성 자료의 일부로서 포함시키는데 적합하다고 생각된다. 이러한 자료는 임상에서 사용하고자 하는 용량으로 적어도 1년간 투여되도록 적절히 설계된 전향적 연구로부터 얻어져야 한다. 1년의 투여기간 동안 어떠한 심각한 약물이상반응이 관찰되지 않으면, 이 환자수는 1년간의 실제 누적 발생률이 3% 이하라는 것을 설명하는데 타당하다.'

In [35]:
one_json_sample['data'][2594]

{'time': '20201029154611',
 'title': '장기투여 의약품의 임상적 안전성 평가 가이드라인',
 'agency': '식품의약품안전처',
 'year': '',
 'content_id': 'PCY_201501080919212741',
 'KDC': '00',
 'paragraphs': [{'context': '5) 흔하지는 않지만, 어떤 약물이상반응은 시간에 따라 빈도나 중증도가 증가할 수 있으며, 어떤 심각한 약물이상반응은 6개월 이상의 투약 이후에만 발생할 수도 있다. 그러므로 일부 환자들은 12개월 동안 약물이 투여되어야 한다. 투여기간과 약물이상반응의 상관성에 대해 충분한 정보가 없을 경우, 1년간 추적할 특정 환자수를 선정하는 것은 대부분, 약물이상반응의 빈도 수준을 발견할 확률과 실제적인 고려사항을 근거로 결정된다. 최소 1년간 약물이 투여된 100명의 환자에서 얻은 자료는 안전성 자료의 일부로서 포함시키는데 적합하다고 생각된다. 이러한 자료는 임상에서 사용하고자 하는 용량으로 적어도 1년간 투여되도록 적절히 설계된 전향적 연구로부터 얻어져야 한다. 1년의 투여기간 동안 어떠한 심각한 약물이상반응이 관찰되지 않으면, 이 환자수는 1년간의 실제 누적 발생률이 3% 이하라는 것을 설명하는데 타당하다.',
   'qas': [{'question': '시간에 따라 빈도나 중증도가 증가하는 것은 뭐야',
     'id': 283269,
     'is_impossible': False,
     'answers': [{'text': '약물이상반응', 'answer_start': 16}]},
    {'question': '빈도나 중증도가 시간에 따라 상승하는 것은 뭐니',
     'id': 283270,
     'is_impossible': False,
     'answers': [{'text': '약물이상반응', 'answer_start': 16}]},
    {'question': '시간이 지날수록 빈도가 감소하는 것이 뭐지

In [31]:
with open(file_name[0], 'r', encoding='utf-8') as one_json_file:
    one_json_sample = json.load(one_json_file)

In [34]:
with open(file_name[0], 'r', encoding='utf-8') as one_json_file:
    one_json_sample = json.load(one_json_file)

source_df = pd.DataFrame(one_json_sample['data'])
source_dict = dict(source_df['paragraphs'].explode())
source_json = pd.json_normalize(source_dict)
source_list = list(source_json.filter(regex='context').values[0])

source_filter_02 = source_json.filter(regex='[0123456789]').values[0]
source_df_02 = pd.DataFrame(source_filter_02)

for source_df_02_value in source_df_02.values:
    for value in source_df_02_value[0]:
        try:
            source_list.append(value['context'])
        except:
            pass

In [51]:
source_list[50]

'5) 흔하지는 않지만, 어떤 약물이상반응은 시간에 따라 빈도나 중증도가 증가할 수 있으며, 어떤 심각한 약물이상반응은 6개월 이상의 투약 이후에만 발생할 수도 있다. 그러므로 일부 환자들은 12개월 동안 약물이 투여되어야 한다. 투여기간과 약물이상반응의 상관성에 대해 충분한 정보가 없을 경우, 1년간 추적할 특정 환자수를 선정하는 것은 대부분, 약물이상반응의 빈도 수준을 발견할 확률과 실제적인 고려사항을 근거로 결정된다. 최소 1년간 약물이 투여된 100명의 환자에서 얻은 자료는 안전성 자료의 일부로서 포함시키는데 적합하다고 생각된다. 이러한 자료는 임상에서 사용하고자 하는 용량으로 적어도 1년간 투여되도록 적절히 설계된 전향적 연구로부터 얻어져야 한다. 1년의 투여기간 동안 어떠한 심각한 약물이상반응이 관찰되지 않으면, 이 환자수는 1년간의 실제 누적 발생률이 3% 이하라는 것을 설명하는데 타당하다.'

In [ ]:
'5) 흔하지는 않지만, 어떤 약물이상반응은 시간에 따라 빈도나 중증도가 증가할 수 있으며, 어떤 심각한 약물이상반응은 6개월 이상의 투약 이후에만 발생할 수도 있다. 그러므로 일부 환자들은 12개월 동안 약물이 투여되어야 한다. 투여기간과 약물이상반응의 상관성에 대해 충분한 정보가 없을 경우, 1년간 추적할 특정 환자수를 선정하는 것은 대부분, 약물이상반응의 빈도 수준을 발견할 확률과 실제적인 고려사항을 근거로 결정된다. 최소 1년간 약물이 투여된 100명의 환자에서 얻은 자료는 안전성 자료의 일부로서 포함시키는데 적합하다고 생각된다. 이러한 자료는 임상에서 사용하고자 하는 용량으로 적어도 1년간 투여되도록 적절히 설계된 전향적 연구로부터 얻어져야 한다. 1년의 투여기간 동안 어떠한 심각한 약물이상반응이 관찰되지 않으면, 이 환자수는 1년간의 실제 누적 발생률이 3% 이하라는 것을 설명하는데 타당하다.'

In [52]:
strange_source = '5) 흔하지는 않지만, 어떤 약물이상반응은 시간에 따라 빈도나 중증도가 증가할 수 있으며, 어떤 심각한 약물이상반응은 6개월 이상의 투약 이후에만 발생할 수도 있다. 그러므로 일부 환자들은 12개월 동안 약물이 투여되어야 한다. 투여기간과 약물이상반응의 상관성에 대해 충분한 정보가 없을 경우, 1년간 추적할 특정 환자수를 선정하는 것은 대부분, 약물이상반응의 빈도 수준을 발견할 확률과 실제적인 고려사항을 근거로 결정된다. 최소 1년간 약물이 투여된 100명의 환자에서 얻은 자료는 안전성 자료의 일부로서 포함시키는데 적합하다고 생각된다. 이러한 자료는 임상에서 사용하고자 하는 용량으로 적어도 1년간 투여되도록 적절히 설계된 전향적 연구로부터 얻어져야 한다. 1년의 투여기간 동안 어떠한 심각한 약물이상반응이 관찰되지 않으면, 이 환자수는 1년간의 실제 누적 발생률이 3% 이하라는 것을 설명하는데 타당하다.'
pprint(strange_source)

('5) 흔하지는 않지만, 어떤 약물이상반응은 시간에 따라 빈도나 중증도가 증가할 수 있으며, 어떤 심각한 약물이상반응은 6개월 이상의 투약 '
 '이후에만 발생할 수도 있다. 그러므로 일부 환자들은 12개월 동안 약물이 투여되어야 한다. 투여기간과 약물이상반응의 상관성에 대해 충분한 '
 '정보가 없을 경우, 1년간 추적할 특정 환자수를 선정하는 것은 대부분, 약물이상반응의 빈도 수준을 발견할 확률과 실제적인 고려사항을 '
 '근거로 결정된다. 최소 1년간 약물이 투여된 100명의 환자에서 얻은 자료는 안전성 자료의 일부로서 포함시키는데 적합하다고 생각된다. '
 '이러한 자료는 임상에서 사용하고자 하는 용량으로 적어도 1년간 투여되도록 적절히 설계된 전향적 연구로부터 얻어져야 한다. 1년의 투여기간 '
 '동안 어떠한 심각한 약물이상반응이 관찰되지 않으면, 이 환자수는 1년간의 실제 누적 발생률이 3% 이하라는 것을 설명하는데 타당하다.')


In [53]:
preprocessing_sentences = formal_preprocessing_text(strange_source)
for preprocessing_sentence in preprocessing_sentences:
    print(preprocessing_sentence)

5) 흔하지는 않지만  어떤 약물이상반응은 시간에 따라 빈도나 중증도가 증가할 수 있으며  어떤 심각한 약물이상반응은 6개월 이상의 투약 이후에만 발생할 수도 있다.
그러므로 일부 환자들은 12개월 동안 약물이 투여되어야 한다.
투여기간과 약물이상반응의 상관성에 대해 충분한 정보가 없을 경우  1년간 추적할 특정 환자수를 선정하는 것은 대부분  약물이상반응의 빈도 수준을 발견할 확률과 실제적인 고려사항을 근거로 결정된다.
최소 1년간 약물이 투여된 100명의 환자에서 얻은 자료는 안전성 자료의 일부로서 포함시키는데 적합하다고 생각된다.
이러한 자료는 임상에서 사용하고자 하는 용량으로 적어도 1년간 투여되도록 적절히 설계된 전향적 연구로부터 얻어져야 한다  1년의 투여기간 동안 어떠한 심각한 약물이상반응이 관찰되지 않으면  이 환자수는 1년간의 실제 누적 발생률이 3  이하라는 것을 설명하는데 타당하다.


In [54]:
print(preprocessing_sentences)

['5) 흔하지는 않지만  어떤 약물이상반응은 시간에 따라 빈도나 중증도가 증가할 수 있으며  어떤 심각한 약물이상반응은 6개월 이상의 투약 이후에만 발생할 수도 있다.', '그러므로 일부 환자들은 12개월 동안 약물이 투여되어야 한다.', '투여기간과 약물이상반응의 상관성에 대해 충분한 정보가 없을 경우  1년간 추적할 특정 환자수를 선정하는 것은 대부분  약물이상반응의 빈도 수준을 발견할 확률과 실제적인 고려사항을 근거로 결정된다.', '최소 1년간 약물이 투여된 100명의 환자에서 얻은 자료는 안전성 자료의 일부로서 포함시키는데 적합하다고 생각된다.', '이러한 자료는 임상에서 사용하고자 하는 용량으로 적어도 1년간 투여되도록 적절히 설계된 전향적 연구로부터 얻어져야 한다  1년의 투여기간 동안 어떠한 심각한 약물이상반응이 관찰되지 않으면  이 환자수는 1년간의 실제 누적 발생률이 3  이하라는 것을 설명하는데 타당하다.']


### AIHUB 법률 규정 (판결서 약관 등) 텍스트 분석 데이터

In [25]:
path_list = ['AIHUB_법률 규정 (판결서 약관 등) 텍스트 분석 데이터/Training/' + '/**/*.json',
             'AIHUB_법률 규정 (판결서 약관 등) 텍스트 분석 데이터/Validation/' + '/**/*.json']
train_file_name, valid_file_name = train_valid_json_file_name_list(path_list)

In [9]:
with open(train_file_name[7000], 'r', encoding='utf-8') as one_json_file:
    one_json_sample = json.load(one_json_file)

In [20]:
train_file_name[7000]

'AIHUB_법률 규정 (판결서 약관 등) 텍스트 분석 데이터/Training\\라벨링데이터\\1.Training\\라벨링데이터\\TL_2.약관\\02.불리\\101_입소,입주,입점계약_가공.json'

In [22]:
file_type_01 = train_file_name[3]
file_type_02 = train_file_name[1021]
file_type_03 = train_file_name[7000]

In [23]:
with open(file_type_01, 'r', encoding='utf-8') as one_json_file:
    one_json_sample = json.load(one_json_file)

    source_df = pd.DataFrame(one_json_sample)
    source_list = source_df.loc['disposalcontent']['disposal'] + \
        source_df.loc['rqestObjet']['mentionedItems'] + \
        source_df.loc['acusrAssrs']['assrs'] + \
        source_df.loc['dedatAssrs']['assrs'] + \
        source_df.loc['bsisFacts']['facts'] + \
        source_df.loc['courtDcss']['dcss'] + \
        source_df.loc['cnclsns']['close']
        
strange_source_list = search_strange_text(source_list)        

In [26]:
train_file_name[3]

'AIHUB_법률 규정 (판결서 약관 등) 텍스트 분석 데이터/Training\\라벨링데이터\\1.Training\\라벨링데이터\\TL_1.판결문\\01.민사\\2018\\2016나686.json'

In [24]:
for strange_source in strange_source_list:
    print(strange_source, end="\n\n")

피고들은 공동하여 원고에게 14,861,556,007원 및 그 중 100,000,000원에 대하여는 2011.5. 7.부터, 10,000,000,000원에 대하여는 2011. 11. 8.부터,15. 10. 23.자 청구취지 및 청구원인신청서 부본 송달 다음 날부터 각 제1심 판결 선고일까지 연6%, 그 다음 날부터 다 갚는 날까지 연 15%의 각 비율로 계산한 돈을 지급한다. 



In [25]:
"피고들은 공동하여 원고에게 14,861,556,007원 및 그 중 100,000,000원에 대하여는 2011.5. 7.부터, 10,000,000,000원에 대하여는 2011. 11. 8.부터,15. 10. 23.자 청구취지 및 청구원인신청서 부본 송달 다음 날부터 각 제1심 판결 선고일까지 연6%, 그 다음 날부터 다 갚는 날까지 연 15%의 각 비율로 계산한 돈을 지급한다. "

'피고들은 공동하여 원고에게 14,861,556,007원 및 그 중 100,000,000원에 대하여는 2011.5. 7.부터, 10,000,000,000원에 대하여는 2011. 11. 8.부터,15. 10. 23.자 청구취지 및 청구원인신청서 부본 송달 다음 날부터 각 제1심 판결 선고일까지 연6%, 그 다음 날부터 다 갚는 날까지 연 15%의 각 비율로 계산한 돈을 지급한다. '

In [26]:
strange_source = "피고들은 공동하여 원고에게 14,861,556,007원 및 그 중 100,000,000원에 대하여는 2011.5. 7.부터, 10,000,000,000원에 대하여는 2011. 11. 8.부터,15. 10. 23.자 청구취지 및 청구원인신청서 부본 송달 다음 날부터 각 제1심 판결 선고일까지 연6%, 그 다음 날부터 다 갚는 날까지 연 15%의 각 비율로 계산한 돈을 지급한다. "
pprint(strange_source)

('피고들은 공동하여 원고에게 14,861,556,007원 및 그 중 100,000,000원에 대하여는 2011.5. 7.부터, '
 '10,000,000,000원에 대하여는 2011. 11. 8.부터,15. 10. 23.자 청구취지 및 청구원인신청서 부본 송달 다음 '
 '날부터 각 제1심 판결 선고일까지 연6%, 그 다음 날부터 다 갚는 날까지 연 15%의 각 비율로 계산한 돈을 지급한다. ')


In [27]:
preprocessing_sentences = formal_preprocessing_text(strange_source)
for preprocessing_sentence in preprocessing_sentences:
    print(preprocessing_sentence)

피고들은 공동하여 원고에게 14 861 556 007원 및 그 중 100 000 000원에 대하여는 2011 5  7 부터  10 000 000 000원에 대하여는 2011  11  8 부터 15  10  23 자 청구취지 및 청구원인신청서 부본 송달 다음 날부터 각 제1심 판결 선고일까지 연6   그 다음 날부터 다 갚는 날까지 연 15 의 각 비율로 계산한 돈을 지급한다.


In [28]:
print(preprocessing_sentences)

['피고들은 공동하여 원고에게 14 861 556 007원 및 그 중 100 000 000원에 대하여는 2011 5  7 부터  10 000 000 000원에 대하여는 2011  11  8 부터 15  10  23 자 청구취지 및 청구원인신청서 부본 송달 다음 날부터 각 제1심 판결 선고일까지 연6   그 다음 날부터 다 갚는 날까지 연 15 의 각 비율로 계산한 돈을 지급한다.']


In [42]:
with open(train_file_name[1141], 'r', encoding='utf-8') as one_json_file:
    one_json_sample = json.load(one_json_file)

source_df = pd.DataFrame.from_dict(one_json_sample, orient='index')
source_df = source_df.transpose()
source_df = pd.DataFrame(one_json_sample)
source_list = [source_df['clauseArticle'][0][0], source_df['comProvision'][0][0]]
        
strange_source_list = search_strange_text(source_list)        

In [43]:
train_file_name[1141]

'AIHUB_법률 규정 (판결서 약관 등) 텍스트 분석 데이터/Training\\라벨링데이터\\1.Training\\라벨링데이터\\TL_2.약관\\01.유리\\037_산후조리원_가공.json'

In [44]:
source_list

['제6조 (이용계약의 변경 및 조정)\n① 이용기간은 13박 14일(2주)를 기본으로 하되, 계약을 체결하는 때에 사업자와 이용자가 상호 협의하여 기간을 조정할 수 있습니다. 다만, 사업자는 고객이 13박 14일보다 단기의 이용을 요청한다는 이유로 계약의 체결을 거절할 수 없습니다.',
 '제6조 (이용계약의 변경 및 조정) \n① 이용기간은 13박 14일(2주)를 기본으로 하되, 계약을 체결하는 때에 사업자와 이용자가 상호 협의하여 기간을 조정할 수 있습니다. 다만, 사업자는 고객이 13박 14일보다 단기의 이용을 요청한다는 이유로 계약의 체결을 거절할 수 없습니다.']

In [30]:
for strange_source in strange_source_list:
    print(strange_source, end="\n\n")

In [ ]:
"제6조 (이용계약의 변경 및 조정)\n① 이용기간은 13박 14일(2주)를 기본으로 하되, 계약을 체결하는 때에 사업자와 이용자가 상호 협의하여 기간을 조정할 수 있습니다. 다만, 사업자는 고객이 13박 14일보다 단기의 이용을 요청한다는 이유로 계약의 체결을 거절할 수 없습니다."


In [28]:
strange_source = "제6조 (이용계약의 변경 및 조정)\n① 이용기간은 13박 14일(2주)를 기본으로 하되, 계약을 체결하는 때에 사업자와 이용자가 상호 협의하여 기간을 조정할 수 있습니다. 다만, 사업자는 고객이 13박 14일보다 단기의 이용을 요청한다는 이유로 계약의 체결을 거절할 수 없습니다."
pprint(strange_source)

('제6조 (이용계약의 변경 및 조정)\n'
 '① 이용기간은 13박 14일(2주)를 기본으로 하되, 계약을 체결하는 때에 사업자와 이용자가 상호 협의하여 기간을 조정할 수 있습니다. '
 '다만, 사업자는 고객이 13박 14일보다 단기의 이용을 요청한다는 이유로 계약의 체결을 거절할 수 없습니다.')


In [29]:
preprocessing_sentences = formal_preprocessing_text(strange_source)
for preprocessing_sentence in preprocessing_sentences:
    print(preprocessing_sentence)

다만  사업자는 고객이 13박 14일보다 단기의 이용을 요청한다는 이유로 계약의 체결을 거절할 수 없습니다.


In [47]:
print(preprocessing_sentences)

['다만  사업자는 고객이 13박 14일보다 단기의 이용을 요청한다는 이유로 계약의 체결을 거절할 수 없습니다.']


In [10]:
with open(train_file_name[7001], 'r', encoding='utf-8') as one_json_file:
    one_json_sample = json.load(one_json_file)

In [17]:
with open(train_file_name[7001], 'r', encoding='utf-8') as one_json_file:
    one_json_sample = json.load(one_json_file)

source_df = pd.DataFrame.from_dict(one_json_sample, orient='index')
source_df = source_df.transpose()
source_list = [source_df['clauseArticle'][0][0], source_df['illdcssBasiss'][0][0],
                source_df['relateLaword'][0][0]]
        
strange_source_list = search_strange_text(source_list)        

In [27]:
train_file_name[7001]

'AIHUB_법률 규정 (판결서 약관 등) 텍스트 분석 데이터/Training\\라벨링데이터\\1.Training\\라벨링데이터\\TL_2.약관\\02.불리\\102_공급계약_가공.json'

In [16]:
source_list

['제 4조 (예약금 및 계약금) \n(1) 예약금은 조리원 이용료의 10퍼센트이며, 예정일로부터 90일 전 취소할 경우, 100퍼센트 환불가능하오니 유념하여 주시기 바랍니다. 전액 환불 가능한 날짜가 지나 환불을 요청하는 경우 남은 기간이 60에서 90일 인 경우 70퍼센트, 30에서 60일인 경우 40퍼센트가 환불되며, 30일 이전인 경우 예약금에 대한 환불은 불가합니다',
 '계약 취소시점과 입소 예정일간의 잔여 기간 동안 대체고객을 충분히 확보할 수 있음에도 불구하고 계약해지로 인한 사업자의 실질적인 손해액을 넘어서는 과도한 위약금을 부과하는 조항은 고객에게 과중한 손해배상의무를 지우는 불공정약관에 해당한다.',
 '제8조(손해배상액의 예정) \n고객에게 부당하게 과중한 지연 손해금 등의 손해배상 의무를 부담시키는 약관 조항은 무효로 한다.']

In [24]:
'제8조(손해배상액의 예정) \n고객에게 부당하게 과중한 지연 손해금 등의 손해배상 의무를 부담시키는 약관 조항은 무효로 한다.'

'제8조(손해배상액의 예정) \n고객에게 부당하게 과중한 지연 손해금 등의 손해배상 의무를 부담시키는 약관 조항은 무효로 한다.'

In [25]:
strange_source = '제8조(손해배상액의 예정) \n고객에게 부당하게 과중한 지연 손해금 등의 손해배상 의무를 부담시키는 약관 조항은 무효로 한다.'
pprint(strange_source)

'제8조(손해배상액의 예정) \n고객에게 부당하게 과중한 지연 손해금 등의 손해배상 의무를 부담시키는 약관 조항은 무효로 한다.'


In [26]:
preprocessing_sentences = formal_preprocessing_text(strange_source)
for preprocessing_sentence in preprocessing_sentences:
    print(preprocessing_sentence)

고객에게 부당하게 과중한 지연 손해금 등의 손해배상 의무를 부담시키는 약관 조항은 무효로 한다.


In [27]:
print(preprocessing_sentences)

['고객에게 부당하게 과중한 지연 손해금 등의 손해배상 의무를 부담시키는 약관 조항은 무효로 한다.']


### AIHUB 일반상식

In [9]:
path_list = ['AIHUB_일반상식/'+ '/**/*.json']
file_name_list = json_file_name_list(path_list)

In [10]:
for file_name in file_name_list:
    if 'ko_wiki_v1_squad' in file_name:
        source_file = file_name

In [11]:
source_file

'AIHUB_일반상식\\ko_wiki_v1_squad.json'

In [12]:
with open(source_file, 'r', encoding='utf-8') as one_json_file:
    one_json_sample = json.load(one_json_file)

source_df = pd.DataFrame(one_json_sample['data'])
source_dict = dict(source_df['paragraphs'].explode())
source_json = pd.json_normalize(source_dict)
source_list = list(source_json.filter(regex='context').values[0])
strange_source_list = search_strange_text(source_list)

In [20]:
one_json_sample['data'][0]['paragraphs']

[{'qas': [{'question': '다테 기미코가 최초로 은퇴 선언을 한게 언제지',
    'answers': [{'answer_start': 260, 'text': '1996년 9월 24일'}],
    'id': '9_f2_wiki_2822-1'}],
  'context': "재팬 오픈에서 4회 우승하였으며, 통산 단식 200승 이상을 거두었다. 1994년 생애 최초로 세계 랭킹 10위권에 진입하였다. 1992년에는 WTA로부터 '올해 가장 많은 향상을 보여준 선수상'(Most Improved Player Of The Year)을 수여받았으며, 일본 남자 패션 협회(Japan Men's Fashion Association)는 그녀를 '가장 패셔너블한 선수'(Most Fashionable)로 칭했다. 생애 두 번째 올림픽 참가 직후인 1996년 9월 24일 최초로 은퇴를 선언하였다. 이후 12년만인 2008년 4월에 예상치 못한 복귀 선언을 하고 투어에 되돌아왔다. 2008년 6월 15일 도쿄 아리아케 인터내셔널 여자 오픈에서 복귀 후 첫 우승을 기록했으며, 2009년 9월 27일에는 한국에서 열린 한솔 코리아 오픈 대회에서 우승하면서 복귀 후 첫 WTA 투어급 대회 우승을 기록했다. 한숨 좀 작작 쉬어!"}]

In [21]:
for i in range(len(one_json_sample['data'])):
    if '~night shift~' in one_json_sample['data'][i]['paragraphs'][0]['context']:
        print(i)

619
34143
42879
64113


In [23]:
one_json_sample['data'][619]

{'paragraphs': [{'qas': [{'question': '나카모리 아키나의 곡 중에 뉴스 프로 엔딩곡으로 쓰인 노래는',
     'answers': [{'answer_start': 218, 'text': 'Rose Bud'}],
     'id': '9_f7_wiki_1037-1'},
    {'question': '밤 어딘가에서 night shift를 발매한 음반사는',
     'answers': [{'answer_start': 189, 'text': 'MCA 빅터'}],
     'id': '9_f7_wiki_1038-1'},
    {'question': '밤 어딘가에서 night shift를 마무리곡으로 사용한 뉴스 이름은',
     'answers': [{'answer_start': 296, 'text': 'NNN 오늘의 사건'}],
     'id': '9_f7_wiki_1039-1'},
    {'question': '사쿠리아 요시코의 직업 알려줘',
     'answers': [{'answer_start': 353, 'text': '앵커'}],
     'id': '9_f7_wiki_1041-1'},
    {'question': '나카모리 아키나의 곡 중에 뉴스 프로그램 엔딩곡으로 만들어진 게 있다던데',
     'answers': [{'answer_start': 1, 'text': '밤 어딘가에서 ~night shift~'}],
     'id': '9_f7_wiki_1043-1'},
    {'question': '밤 어딘가에서 night shift를 낸 음반사는',
     'answers': [{'answer_start': 189, 'text': 'MCA 빅터'}],
     'id': '9_f7_wiki_1044-1'},
    {'question': '밤 어딘가에서 night shift를 엔딩곡으로 쓴 뉴스는',
     'answers': [{'answer_start': 296, 'text': 'NNN 오늘의 

In [22]:
one_json_sample['data'][619]['paragraphs'][0]['context']

'〈밤 어딘가에서 ~night shift~〉(일본어: 夜のどこかで 〜night shift〜 요루노도코카데 나이트시프트)는 일본의 여가수 나카모리 아키나(中森明菜)의 29번째 싱글로 1994년 9월 2일에 발매하였다. (8cmCD: MVDD-10007) 작사는 나츠노 세리코(夏野芹子)가, 작곡과 편곡은 고토 츠구토시(後藤次利)가 담당하였고 MCA 빅터의 레이블로 출시되었다. B사이드 곡은 〈Rose Bud〉로 A사이드와 마찬가지로 작사는 나츠노 세리코가, 작곡과 편곡은 역시 고토 츠구토시가 도맡았다. 이 곡은 뉴스 프로그램인 《NNN 오늘의 사건》(NNNきょうの出来事)의 엔딩곡으로 제작되었는데 디스크 재킷 역시 뉴스 프로그램의 앵커인 사쿠라이 요시코와 촬영하였다. 나카모리는 뉴스 프로그램의 주제가로 사용될 것이라는 말을 듣고 어떤 목소리로 노래해야 할지 고민했지만 뉴스에 등장하는 여성들을 향한 응원가로 생각하면서 불렀다고 밝혔다. 1994년 9월 12일, 오리콘 주간 싱글 차트 14위로 처음 등장하여 차트 톱100을 총 7주간 지켰다.'

In [11]:
for strange_source in strange_source_list:
    print(strange_source, end="\n\n")

일반성면은 동부 5개 면의 교통, 문화, 교육, 상업의 중심지로서 일찍부터 상업이 발달한 곳으로 날로 반성 재래시장이 번성하고 있는 고장이다. 면적은 19.41 km²로 진주시 16개 읍면동 중 가장 적은 면이지만 인근에 경남 산림환경연구원이 있어 그곳을 찾는 관광객이 이곳 일반성면을 지나간다. 2012년 1월 1일 기준으로 인구 3,233명 (남 : 1,556명, 여 : 1,677명) 1,413세대로 6개 법정리 19개 자연마을 31개 반으로 구성되어 있다. [1]

《Remapping The Human Soul》은 전작들과는 달리 무겁고 음침한 분위기에서 트랙을 이끌어간다. DJ 투컷이 프로듀싱을 담당한 1CD The Brain은 정부와 신에 대한 불신("Nocturne", 〈희생양〉), 살인을 저지른 남자의 이야기(〈피해망상 Pt.1〉) 안락사로 유명한 잭 케보키언에 대한 부제가 붙은 곡("Mr.Doctor") 등 대중적으로 다루기 무거운 곡들이 배치되어있고, 타블로가 담당한 2CD The Heart는 버림받은 로봇들의 이야기("Broken Toys"), 타블로와 미쓰라 진의 랩이 겹쳐져 이루어진 자살 암시(〈행복합니다〉), 안티팬들을 겨냥한 신랄한 욕설("FAQ")이 담기는 등 전작 《Swan Songs》에서 나타나지 않았던 암흑적이고 어두운 느낌이 음반 전체를 지배한다. 그러면서 〈Fan〉, 〈Love Love Love〉 등에서 중독성 있는 멜로디로 보여준 팝(대중음악)과의 성공적인 조화는 일반 대중이 듣기에 산뜻한 분위기를 담고 있다. 2007년 1월 23일에 음반을 발매하자마자 타이틀 곡인 〈Fan〉과 후속곡인 〈Love Love Love〉는 음악 차트 상위권을 기록하였다. 2월 3일에는 타이틀 곡 〈Fan〉이 음반 발매 10일 만에 벅스 차트 1위를 기록하였고, 2월 6일에는 쥬크온 차트 1위를 기록하는 등 각종 음반 판매량과 온라인 음악차트에서 1위를 석권하였다. 《Remapping The Human Soul》는 2007년 1월 음반 판매량 순

In [12]:
"〈밤 어딘가에서 ~night shift~〉(일본어: 夜のどこかで 〜night shift〜 요루노도코카데 나이트시프트)는 일본의 여가수 나카모리 아키나(中森明菜)의 29번째 싱글로 1994년 9월 2일에 발매하였다. (8cmCD: MVDD-10007) 작사는 나츠노 세리코(夏野芹子)가, 작곡과 편곡은 고토 츠구토시(後藤次利)가 담당하였고 MCA 빅터의 레이블로 출시되었다. B사이드 곡은 〈Rose Bud〉로 A사이드와 마찬가지로 작사는 나츠노 세리코가, 작곡과 편곡은 역시 고토 츠구토시가 도맡았다. 이 곡은 뉴스 프로그램인 《NNN 오늘의 사건》(NNNきょうの出来事)의 엔딩곡으로 제작되었는데 디스크 재킷 역시 뉴스 프로그램의 앵커인 사쿠라이 요시코와 촬영하였다. 나카모리는 뉴스 프로그램의 주제가로 사용될 것이라는 말을 듣고 어떤 목소리로 노래해야 할지 고민했지만 뉴스에 등장하는 여성들을 향한 응원가로 생각하면서 불렀다고 밝혔다. 1994년 9월 12일, 오리콘 주간 싱글 차트 14위로 처음 등장하여 차트 톱100을 총 7주간 지켰다."

'〈밤 어딘가에서 ~night shift~〉(일본어: 夜のどこかで 〜night shift〜 요루노도코카데 나이트시프트)는 일본의 여가수 나카모리 아키나(中森明菜)의 29번째 싱글로 1994년 9월 2일에 발매하였다. (8cmCD: MVDD-10007) 작사는 나츠노 세리코(夏野芹子)가, 작곡과 편곡은 고토 츠구토시(後藤次利)가 담당하였고 MCA 빅터의 레이블로 출시되었다. B사이드 곡은 〈Rose Bud〉로 A사이드와 마찬가지로 작사는 나츠노 세리코가, 작곡과 편곡은 역시 고토 츠구토시가 도맡았다. 이 곡은 뉴스 프로그램인 《NNN 오늘의 사건》(NNNきょうの出来事)의 엔딩곡으로 제작되었는데 디스크 재킷 역시 뉴스 프로그램의 앵커인 사쿠라이 요시코와 촬영하였다. 나카모리는 뉴스 프로그램의 주제가로 사용될 것이라는 말을 듣고 어떤 목소리로 노래해야 할지 고민했지만 뉴스에 등장하는 여성들을 향한 응원가로 생각하면서 불렀다고 밝혔다. 1994년 9월 12일, 오리콘 주간 싱글 차트 14위로 처음 등장하여 차트 톱100을 총 7주간 지켰다.'

In [13]:
strange_source = "〈밤 어딘가에서 ~night shift~〉(일본어: 夜のどこかで 〜night shift〜 요루노도코카데 나이트시프트)는 일본의 여가수 나카모리 아키나(中森明菜)의 29번째 싱글로 1994년 9월 2일에 발매하였다. (8cmCD: MVDD-10007) 작사는 나츠노 세리코(夏野芹子)가, 작곡과 편곡은 고토 츠구토시(後藤次利)가 담당하였고 MCA 빅터의 레이블로 출시되었다. B사이드 곡은 〈Rose Bud〉로 A사이드와 마찬가지로 작사는 나츠노 세리코가, 작곡과 편곡은 역시 고토 츠구토시가 도맡았다. 이 곡은 뉴스 프로그램인 《NNN 오늘의 사건》(NNNきょうの出来事)의 엔딩곡으로 제작되었는데 디스크 재킷 역시 뉴스 프로그램의 앵커인 사쿠라이 요시코와 촬영하였다. 나카모리는 뉴스 프로그램의 주제가로 사용될 것이라는 말을 듣고 어떤 목소리로 노래해야 할지 고민했지만 뉴스에 등장하는 여성들을 향한 응원가로 생각하면서 불렀다고 밝혔다. 1994년 9월 12일, 오리콘 주간 싱글 차트 14위로 처음 등장하여 차트 톱100을 총 7주간 지켰다."
pprint(strange_source)

('〈밤 어딘가에서 ~night shift~〉(일본어: 夜のどこかで 〜night shift〜 요루노도코카데 나이트시프트)는 일본의 여가수 '
 '나카모리 아키나(中森明菜)의 29번째 싱글로 1994년 9월 2일에 발매하였다. (8cmCD: MVDD-10007) 작사는 나츠노 '
 '세리코(夏野芹子)가, 작곡과 편곡은 고토 츠구토시(後藤次利)가 담당하였고 MCA 빅터의 레이블로 출시되었다. B사이드 곡은 〈Rose '
 'Bud〉로 A사이드와 마찬가지로 작사는 나츠노 세리코가, 작곡과 편곡은 역시 고토 츠구토시가 도맡았다. 이 곡은 뉴스 프로그램인 《NNN '
 '오늘의 사건》(NNNきょうの出来事)의 엔딩곡으로 제작되었는데 디스크 재킷 역시 뉴스 프로그램의 앵커인 사쿠라이 요시코와 촬영하였다. '
 '나카모리는 뉴스 프로그램의 주제가로 사용될 것이라는 말을 듣고 어떤 목소리로 노래해야 할지 고민했지만 뉴스에 등장하는 여성들을 향한 '
 '응원가로 생각하면서 불렀다고 밝혔다. 1994년 9월 12일, 오리콘 주간 싱글 차트 14위로 처음 등장하여 차트 톱100을 총 7주간 '
 '지켰다.')


In [14]:
preprocessing_sentences = formal_preprocessing_text(strange_source)
for preprocessing_sentence in preprocessing_sentences:
    print(preprocessing_sentence)

B사이드 곡은  Rose Bud 로 A사이드와 마찬가지로 작사는 나츠노 세리코가  작곡과 편곡은 역시 고토 츠구토시가 도맡았다.
이 곡은 뉴스 프로그램인  NNN 오늘의 사건 (NNN    出来事)의 엔딩곡으로 제작되었는데 디스크 재킷 역시 뉴스 프로그램의 앵커인 사쿠라이 요시코와 촬영하였다.
나카모리는 뉴스 프로그램의 주제가로 사용될 것이라는 말을 듣고 어떤 목소리로 노래해야 할지 고민했지만 뉴스에 등장하는 여성들을 향한 응원가로 생각하면서 불렀다고 밝혔다.
1994년 9월 12일  오리콘 주간 싱글 차트 14위로 처음 등장하여 차트 톱100을 총 7주간 지켰다.


In [15]:
print(preprocessing_sentences)

['B사이드 곡은  Rose Bud 로 A사이드와 마찬가지로 작사는 나츠노 세리코가  작곡과 편곡은 역시 고토 츠구토시가 도맡았다.', '이 곡은 뉴스 프로그램인  NNN 오늘의 사건 (NNN    出来事)의 엔딩곡으로 제작되었는데 디스크 재킷 역시 뉴스 프로그램의 앵커인 사쿠라이 요시코와 촬영하였다.', '나카모리는 뉴스 프로그램의 주제가로 사용될 것이라는 말을 듣고 어떤 목소리로 노래해야 할지 고민했지만 뉴스에 등장하는 여성들을 향한 응원가로 생각하면서 불렀다고 밝혔다.', '1994년 9월 12일  오리콘 주간 싱글 차트 14위로 처음 등장하여 차트 톱100을 총 7주간 지켰다.']
